In [115]:
import os
import pandas as pd
import gc
mode = 'online' # offline/online: offline validation or online submission
now_phase = 5

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM
from deepctr.models.din import DIN
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names, VarLenSparseFeat
from sklearn.metrics import log_loss, roc_auc_score
from tensorflow.python.keras.models import Model, load_model, save_model
from deepctr.layers import custom_objects
import tensorflow as tf
tf.random.set_seed(1234)

In [116]:
offline_answer_path = 'offline_underexpose_answer_2'
offline_test_path = 'offline_underexpose_test_2' 
offline_train_path = 'offline_underexpose_train_2'

train_path = 'underexpose_train' if mode == 'online' else offline_train_path
test_path = 'underexpose_test' if mode == 'online' else offline_test_path

output_path = 'sub_{}'.format(mode)
if not os.path.exists(output_path): os.mkdir(output_path)

In [4]:
drive_path  = 'debiasing'

## Data Split

In [5]:
# create offline val data
import pandas as pd  
import numpy as np
import os

sample_user_num = 1600
if not os.path.exists(offline_answer_path): os.mkdir(offline_answer_path)
if not os.path.exists(offline_test_path): os.mkdir(offline_test_path)
if not os.path.exists(offline_train_path): os.mkdir(offline_train_path)
np.random.seed(1234)

for phase in range(now_phase+1):
    click_train = pd.read_csv('underexpose_train/underexpose_train_click-{}.csv'.format(phase), header=None,
                            names=['user_id', 'item_id', 'time'])
    all_user_ids = click_train['user_id'].unique()
  
    sample_user_ids = np.random.choice(all_user_ids, size=1600, replace=False)

    click_test = click_train[click_train['user_id'].isin(sample_user_ids)]
    click_train = click_train[~click_train['user_id'].isin(sample_user_ids)]

    click_test = click_test.sort_values(by=['user_id', 'time'])
    click_answer = click_test.groupby('user_id').tail(1)
    click_test = click_test.groupby('user_id').apply(lambda x:x[:-1]).reset_index(drop=True)
    click_answer = click_answer[click_answer['user_id'].isin(click_test['user_id'].unique())] # 防止有些用户只有1个点击数据，去掉
    click_test = click_test[click_test['user_id'].isin(click_answer['user_id'].unique())]
    click_qtime = click_answer[['user_id', 'time']]

    click_train.to_csv(offline_train_path + '/underexpose_train_click-{}.csv'.format(phase), index=False, header=None)
    click_answer.to_csv(offline_answer_path + '/underexpose_test_qtime_with_answer-{}.csv'.format(phase), index=False, header=None)
    click_test.to_csv(offline_test_path + '/underexpose_test_click-{}.csv'.format(phase), index=False, header=None)
    click_qtime.to_csv(offline_test_path + '/underexpose_test_qtime-{}.csv'.format(phase), index=False, header=None)

## Common

In [6]:
import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import math  
import numpy as np
# click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(0), header=None,
                                  # names=['user_id', 'item_id', 'time'])
# click_train = click_train.sort_values(by=['user_id', 'time'])
# 6789
user_feat_df = pd.read_csv('underexpose_train/underexpose_user_feat.csv',header=None, names=['user_id','age_level','gender','city_level'])
user_feat_df = user_feat_df.drop_duplicates('user_id')
user_feat_df['age_level'].fillna("-1", inplace=True) # user_feat_df['age_level'].value_counts().index[0]
user_feat_df['gender'].fillna("-1", inplace=True) # user_feat_df['gender'].value_counts().index[0]
user_feat_df['city_level'].fillna("-1", inplace=True) # user_feat_df['city_level'].value_counts().index[0]


item_feat_cols = ['item_id',] + ['txt_embed_'+ str(i) for i in range(128)] + ['img_embed_'+ str(i) for i in range(128)]
item_feat_df = pd.read_csv('underexpose_train/underexpose_item_feat.csv',header=None, names=item_feat_cols)
item_feat_df['txt_embed_0'] = item_feat_df['txt_embed_0'].apply(lambda x:float(x[1:]))
item_feat_df['txt_embed_127'] = item_feat_df['txt_embed_127'].apply(lambda x:float(x[:-1]))
item_feat_df['img_embed_0'] = item_feat_df['img_embed_0'].apply(lambda x:float(x[1:]))
item_feat_df['img_embed_127'] = item_feat_df['img_embed_127'].apply(lambda x:float(x[:-1]))

In [7]:
online_total_click = pd.DataFrame()
for c in range(now_phase + 1):
    print('phase:', c)
    click_train = pd.read_csv('underexpose_train/underexpose_train_click-{}.csv'.format(c), header=None,
                              names=['user_id', 'item_id', 'time'])
    click_test = pd.read_csv('underexpose_test/underexpose_test_click-{}.csv'.format(c), header=None,
                              names=['user_id', 'item_id', 'time'])

    all_click = click_train.append(click_test)
    all_click['phase'] = c
    online_total_click = online_total_click.append(all_click)
    
online_top50_click_np = online_total_click['item_id'].value_counts().index[:50].values
online_top50_click = ','.join([str(i) for i in online_top50_click_np])

phase: 0
phase: 1
phase: 2
phase: 3
phase: 4
phase: 5


In [8]:
total_click = pd.DataFrame()
for c in range(now_phase + 1):
    print('phase:', c)
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,
                              names=['user_id', 'item_id', 'time'])
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,
                              names=['user_id', 'item_id', 'time'])

    all_click = click_train.append(click_test)
    all_click['phase'] = c
    total_click = total_click.append(all_click)
offline_top50_click_np = total_click['item_id'].value_counts().index[:50].values
offline_top50_click = ','.join([str(i) for i in offline_top50_click_np])

phase: 0
phase: 1
phase: 2
phase: 3
phase: 4
phase: 5


In [9]:
# fill user to 50 items  
def get_predict(df, pred_col, top_fill):
    top_fill = [int(t) for t in top_fill.split(',')]
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]
    ids = list(df['user_id'].unique())
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])
    fill_df.sort_values('user_id', inplace=True)
    fill_df['item_id'] = top_fill * len(ids)
    fill_df[pred_col] = scores * len(ids)
    print(len(fill_df))
    df = df.append(fill_df)
    df.sort_values(pred_col, ascending=False, inplace=True)
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)
    df = df[df['rank'] <= 50]
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',',
                                                                                                   expand=True).reset_index()
    return df

In [10]:
def get_feat_topk_click_df(whole_click, user_feat_df, feat_cols=['gender','age_level']):
   whole_click_user_feat_df = pd.merge(user_feat_df, whole_click, how='inner', on='user_id')
   whole_click_feat_topk_df = whole_click_user_feat_df.groupby(feat_cols+ ['item_id']).size().reset_index().rename(columns={0:'click_num'})
   whole_click_feat_topk_df['rank'] = whole_click_feat_topk_df.groupby(feat_cols)['click_num'].rank(method='first', ascending=False)
   whole_click_feat_topk_df = whole_click_feat_topk_df[whole_click_feat_topk_df['rank'] <= 50]
   whole_click_feat_topk_df = whole_click_feat_topk_df.sort_values(by=feat_cols+['rank',])
   whole_click_feat_topk_df['sim'] = whole_click_feat_topk_df['rank'].apply(lambda x: -x)
   return whole_click_feat_topk_df

def get_reco_df_fill_topk(recom_df, user_feat_df, whole_click_user_feat_topk_df, global_top_click_items, feat_cols=['gender','age_level']):
   # rec_users in the user_feat_df
   existing_users = set(recom_df['user_id']) & set(user_feat_df['user_id'])
   recommend_fill_df = pd.merge(user_feat_df[user_feat_df['user_id'].isin(existing_users)], whole_click_user_feat_topk_df, 
                                on=feat_cols, how='inner')[['user_id', 'item_id', 'sim']]
   existing_users = set(recommend_fill_df['user_id'])
   print(len(existing_users), len(existing_users)*50, len(recommend_fill_df))

   # rec_users not in user_feat_df
   all_users = set(recom_df['user_id'])
   top_fill = [int(t) for t in global_top_click_items.split(',')]
   scores = [-1 * i for i in range(51, len(top_fill) + 51)]
   ids = list(all_users)
   all_fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])
   all_fill_df.sort_values('user_id', inplace=True)
   all_fill_df['item_id'] = top_fill * len(ids)
   all_fill_df['sim'] = scores * len(ids)
   print(len(all_users), len(all_users)*50, len(all_fill_df))

   recommend_fill_df = recommend_fill_df.append(all_fill_df)
   print(len(recommend_fill_df))
   return recommend_fill_df


def get_feat_predict(rec_df, whole_click_df, user_feat_df, global_top_click_items, feat_cols=['gender','age_level']):
    whole_click_feat_topk_df = get_feat_topk_click_df(whole_click_df, user_feat_df, feat_cols)
    recommend_fill_df = get_reco_df_fill_topk(rec_df, user_feat_df, whole_click_feat_topk_df, global_top_click_items, feat_cols)
    
    df = rec_df.append(recommend_fill_df)
    df.sort_values('sim', ascending=False, inplace=True)
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')
    df['rank'] = df.groupby('user_id')['sim'].rank(method='first', ascending=False)
    df = df[df['rank'] <= 50]
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',',
                                                                                             expand=True).reset_index()
    return df

In [11]:
from collections import defaultdict

def make_user_time_tuple(group_df, user_col='user_id', item_col='item_id', time_col='time'):
    user_time_tuples = list(zip(group_df[user_col], group_df[time_col]))
    return user_time_tuples

def make_item_time_tuple(group_df, user_col='user_id', item_col='item_id', time_col='time'):
  # group_df = group_df.drop_duplicates(subset=[user_col, item_col], keep='last')
  item_time_tuples = list(zip(group_df[item_col], group_df[time_col]))
  return item_time_tuples

def get_user_item_time_dict(df, user_col='user_id', item_col='item_id', time_col='time'):
    user_item_ = df.sort_values(by=[user_col, time_col])
    user_item_ = user_item_.groupby(user_col).apply(lambda group: make_item_time_tuple(group, user_col, item_col, time_col)).reset_index().rename(columns={0: 'item_id_time_list'})
    user_item_time_dict = dict(zip(user_item_[user_col], user_item_['item_id_time_list']))
    return user_item_time_dict

def get_item_user_time_dict(df, user_col='user_id', item_col='item_id', time_col='time'):
    item_user_df = df.sort_values(by=[item_col, time_col])
    item_user_df = item_user_df.groupby(item_col).apply(
        lambda group: make_user_time_tuple(group, user_col, item_col, time_col)).reset_index().rename(
        columns={0: 'user_id_time_list'})
    item_user_time_dict = dict(zip(item_user_df[item_col], item_user_df['user_id_time_list']))
    return item_user_time_dict

def get_user_min_time_dict(df,  user_col='user_id', item_col='item_id', time_col='time'):
    df = df.sort_values(by=[user_col, time_col])
    df = df.groupby(user_col).head(1)
    user_min_time_dict = dict(zip(df[user_col], df[time_col]))
    return user_min_time_dict


def item_based_recommend(sim_item_corr, user_item_time_dict, user_id, top_k, item_num, alpha=15000):
    rank = {}
    if user_id not in user_item_time_dict:
      return []
    interacted_item_times = user_item_time_dict[user_id]
    min_time = min([time for item, time in interacted_item_times])
    interacted_items = set([item for item, time in interacted_item_times])
    
    miss_item_num = 0
    for loc, (i, time) in enumerate(interacted_item_times):
        if i not in sim_item_corr:
          miss_item_num += 1
          continue
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda x: x[1], reverse=True)[0:top_k]:
            if j not in interacted_items:
                rank.setdefault(j, 0)

                content_weight = 1.0
                if item_content_sim_dict.get(i, {}).get(j, None) is not None:
                  content_weight += item_content_sim_dict[i][j]
                if item_content_sim_dict.get(j, {}).get(i, None) is not None:
                  content_weight += item_content_sim_dict[j][i]

                time_weight = np.exp(alpha*(time - min_time))
                loc_weight = (0.9**(len(interacted_item_times)-loc)) 
                rank[j] += loc_weight * time_weight * content_weight * wij 
    if miss_item_num > 10:     
        print('user_id={}, miss_item_num={}'.format(user_id, miss_item_num))
    sorted_rank_items = sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]
    
    return sorted_rank_items


def user_based_recommend(sim_user_corr, user_item_time_dict, user_id, top_k, item_num, alpha=15000):
    rank = {}
    interacted_items = set([i for i, t in user_item_time_dict[user_id]]) 
    interacted_item_time_list = user_item_time_dict[user_id]
    interacted_num = len(interacted_items)

    min_time = min([t for i,t in interacted_item_time_list])
    time_weight_dict = {i: np.exp(alpha*(t-min_time)) for i,t in interacted_item_time_list}
    loc_weight_dict = {i: 0.9**(interacted_num-loc) for loc, (i,t) in enumerate(interacted_item_time_list)}

    for sim_v, wuv in sorted(sim_user_corr[user_id].items(), key=lambda x:x[1], reverse=True)[0:top_k]:
      for j, j_time in user_item_time_dict[sim_v]:
        if j not in interacted_items:
          rank.setdefault(j, 0)

          content_weight = 1.0
          for loc, (i, t) in enumerate(interacted_item_time_list):
              loc_weight = loc_weight_dict[i]
              time_weight = time_weight_dict[i]
              if item_content_sim_dict.get(i, {}).get(j, None) is not None:
                content_weight += time_weight*loc_weight*item_content_sim_dict[i][j]

          # weight = np.exp(-15000*abs(j_time-q_time))
          rank[j] += content_weight * wuv 

    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]

## content

In [12]:
import collections
import pickle
import os
def get_content_sim_item(item_feat_df, topk=100):
   sim_path = os.path.join(drive_path, 'item_content_sim_dict.pkl')
   if os.path.exists(sim_path):
      with open(sim_path, 'rb') as f:
        return pickle.load(f)

   item_idx_2_rawid_dict = dict(zip(item_feat_df.index, item_feat_df['item_id']))
   txt_item_feat_df = item_feat_df.filter(regex="txt*")
   img_item_feat_df = item_feat_df.filter(regex="img*") 

   txt_item_feat_np = np.ascontiguousarray(txt_item_feat_df.values, dtype=np.float32)
   img_item_feat_np = np.ascontiguousarray(img_item_feat_df.values, dtype=np.float32)
   
   # norm
   txt_item_feat_np = txt_item_feat_np / np.linalg.norm(txt_item_feat_np, axis=1, keepdims=True)
   img_item_feat_np = img_item_feat_np / np.linalg.norm(img_item_feat_np, axis=1, keepdims=True)

   import faiss    
   txt_index = faiss.IndexFlatIP(128)
   txt_index.add(txt_item_feat_np)

   img_index = faiss.IndexFlatIP(128)
   img_index.add(img_item_feat_np)

   item_sim_dict = collections.defaultdict(dict)

   def search(feat_index, feat_np):
      sim, idx = feat_index.search(feat_np, topk)
      for target_idx, sim_value_list, rele_idx_list in zip(range(len(feat_np)), sim, idx):
          target_raw_id = item_idx_2_rawid_dict[target_idx]
          for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1: ]):
            rele_raw_id = item_idx_2_rawid_dict[rele_idx]
            item_sim_dict[target_raw_id][rele_raw_id] = item_sim_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value
   
   search(txt_index, txt_item_feat_np)
   search(img_index, img_item_feat_np)

   with open(sim_path, 'wb') as f:
    pickle.dump(item_sim_dict, f)

   return item_sim_dict

## swing

In [13]:
def swing(df, user_col='user_id', item_col='item_id', time_col='time'):
    # 1. item, (u1,t1), (u2, t2).....
    item_user_df = df.sort_values(by=[item_col, time_col])
    item_user_df = item_user_df.groupby(item_col).apply(
        lambda group: make_user_time_tuple(group, user_col, item_col, time_col)).reset_index().rename(
        columns={0: 'user_id_time_list'})
    item_user_time_dict = dict(zip(item_user_df[item_col], item_user_df['user_id_time_list']))

    user_item_time_dict = defaultdict(list)
    # 2. ((u1, u2), i1, d12)
    u_u_cnt = defaultdict(list)
    item_cnt = defaultdict(int)
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        for u, u_time in user_time_list:
            # just record
            item_cnt[item] += 1
            user_item_time_dict[u].append((item, u_time))

            for relate_u, relate_u_time in user_time_list:
                if relate_u == u:
                    continue
               
                key = (u, relate_u)  if u <= relate_u else (relate_u, u)
                u_u_cnt[key].append((item, np.abs(u_time - relate_u_time)))


    # 3. (i1,i2), sim
    sim_item = {}
    alpha = 5.0
    for u_u, co_item_times in u_u_cnt.items():
        num_co_items = len(co_item_times)
        for i, i_time_diff in co_item_times:
            sim_item.setdefault(i, {})
            for j, j_time_diff in co_item_times:
              if j == i:
                continue
              weight = 1.0 # np.exp(-15000*(i_time_diff + j_time_diff))
              sim_item[i][j] = sim_item[i].setdefault(j, 0.) + weight / (alpha + num_co_items)
    # 4. norm by item count
    sim_item_corr = sim_item.copy()
    for i, related_items in sim_item.items():
        for j, cij in related_items.items():
            sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i] * item_cnt[j])
       
    return sim_item_corr, user_item_time_dict

## time-dir-aware itemcf

In [14]:
def get_time_dir_aware_sim_item(df, user_col='user_id', item_col='item_id', time_col='time'):
    user_item_time_dict = get_user_item_time_dict(df, user_col, item_col, time_col)

    sim_item = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        for loc_1, (i, i_time) in enumerate(item_time_list):
            item_cnt[i] += 1
            sim_item.setdefault(i, {})
            for loc_2, (relate_item, related_time) in enumerate(item_time_list):
                if i == relate_item:
                    continue
                loc_alpha = 1.0 if loc_2 > loc_1 else 0.7
                loc_weight = loc_alpha * (0.8**(np.abs(loc_2-loc_1)-1)) 
                time_weight = np.exp(-15000*np.abs(i_time-related_time))

                sim_item[i].setdefault(relate_item, 0)
                sim_item[i][relate_item] += loc_weight * time_weight / math.log(1 + len(item_time_list))
                
    sim_item_corr = sim_item.copy()
    for i, related_items in tqdm(sim_item.items()):
        for j, cij in related_items.items():
            sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i] * item_cnt[j])
            # sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i]*item_cnt[j])+cij/min(item_cnt[i], item_cnt[j])+0.5*cij/(item_cnt[i]+item_cnt[j])

    return sim_item_corr, user_item_time_dict


## bi-graph

In [15]:
def get_bi_sim_item(df, user_col='user_id', item_col='item_id', time_col='time'):
    item_user_time_dict = get_item_user_time_dict(df, user_col, item_col, time_col)
    user_item_time_dict = get_user_item_time_dict(df, user_col, item_col, time_col)

    item_cnt = defaultdict(int)  
    for user, item_times in tqdm(user_item_time_dict.items()):  
        for i,t in item_times:  
            item_cnt[i] += 1  

    sim_item = {}
    
    for item, user_time_lists in tqdm(item_user_time_dict.items()):
    
        sim_item.setdefault(item, {}) 
    
        for u, item_time in user_time_lists:
        
            tmp_len = len(user_item_time_dict[u])
        
            for relate_item, related_time in user_item_time_dict[u]:
                sim_item[item].setdefault(relate_item, 0)
                weight = np.exp(-15000*np.abs(related_time - item_time))
                sim_item[item][relate_item] += weight / (math.log(len(user_time_lists)+1) * math.log(tmp_len+1))
       
    return sim_item, user_item_time_dict

## user-cf

In [16]:
# user-cf
def get_sim_user(df, user_col='user_id', item_col='item_id', time_col='time'):
    # user_min_time_dict = get_user_min_time_dict(df, user_col, item_col, time_col) # user first time 
    # history
    user_item_time_dict = get_user_item_time_dict(df)
    # item, [u1, u2, ...,]
    item_user_time_dict = get_item_user_time_dict(df)

    sim_user = {}
    user_cnt = defaultdict(int)
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        num_users = len(user_time_list)
        for u, t in user_time_list:
            user_cnt[u] += 1
            sim_user.setdefault(u, {})
            for relate_user, relate_t in user_time_list:
                # time_diff_relate_u = 1.0/(1.0+10000*abs(relate_t-t))
                if u == relate_user:
                    continue
                sim_user[u].setdefault(relate_user, 0)
                weight = 1.0
                sim_user[u][relate_user] += weight / math.log(1 + num_users) # 流行度高的衰减

    sim_user_corr = sim_user.copy()
    for u, related_users in tqdm(sim_user.items()):
        for v, cuv in related_users.items():
            sim_user_corr[u][v] = cuv / math.sqrt(user_cnt[u] * user_cnt[v])

    return sim_user_corr, user_item_time_dict


## recall process

1. 划分history和last
2. history计算相似性
3. 召回

In [17]:
topk_num = 200
recommend_num = 800

### recall one source

In [18]:
# 基于计算的相似性汇总
def norm_recall_item_score_list(sorted_recall_item_list):
    if len(sorted_recall_item_list) == 0: return sorted_recall_item_list
    
    assert sorted_recall_item_list[0][1] >= sorted_recall_item_list[-1][1] # 稍微check下是否排序的
    max_sim = sorted_recall_item_list[0][1]
    min_sim = sorted_recall_item_list[-1][1]
    
    norm_sorted_recall_item_list = []
    for item, score in sorted_recall_item_list:
        if max_sim > 0:
            norm_score = 1.0 * (score - min_sim) / (max_sim - min_sim) if max_sim > min_sim else 1.0
        else:
            norm_score = 0.0 # topk-fill set to 0.0
        norm_sorted_recall_item_list.append((item, norm_score))
    return norm_sorted_recall_item_list


def norm_user_recall_item_dict(recall_item_dict):
    norm_recall_item_dict = {}
    for u, sorted_recall_item_list in recall_item_dict.items():
        norm_recall_item_dict[u] = norm_recall_item_score_list(sorted_recall_item_list)
    return norm_recall_item_dict


def get_recall_results(item_sim_dict, user_item_dict, target_user_ids=None, item_based=True):
    if target_user_ids is None:
        target_user_ids = user_item_dict.keys()
    recall_item_dict = {}
    
    top50_click_np = offline_top50_click_np if mode == 'offline' else online_top50_click_np
    for u in tqdm(target_user_ids):
        if item_based:
            recall_items = item_based_recommend(item_sim_dict, user_item_dict, u, recommend_num, topk_num)
        else:
            recall_items = user_based_recommend(item_sim_dict, user_item_dict, u, recommend_num, topk_num)
        
        if len(recall_items) == 0: 
            recall_items = [(top50_click_np[0], 0.0)] # 防止该用户丢失
        
        recall_item_dict[u] = recall_items
        
    return recall_item_dict


### aggregate multi-recall sources

In [19]:
# item-cf
# bi-graph
# user-cf
# item-cf
def agg_recall_results(recall_item_dict_list, is_norm=True, ret_type='tuple'):
    print('aggregate recall results begin....')
    agg_recall_item_dict = {}
    for recall_item_dict in recall_item_dict_list:
        if is_norm:
            recall_item_dict = norm_user_recall_item_dict(recall_item_dict)
        for u, recall_items in recall_item_dict.items():
            agg_recall_item_dict.setdefault(u, {})
            for i, score in recall_items:
                agg_recall_item_dict[u].setdefault(i, 0.0)
                agg_recall_item_dict[u][i] += score # 累加
                
    if ret_type == 'tuple':
        agg_recall_item_tuple_dict = {}
        for u, recall_item_dict in agg_recall_item_dict.items():
            sorted_recall_item_tuples = sorted(recall_item_dict.items(), key=lambda x:x[1], reverse=True)
            agg_recall_item_tuple_dict[u] = sorted_recall_item_tuples
        return agg_recall_item_tuple_dict
    
    if ret_type == 'df':
        recall_u_i_score_pair_list = []
        for u, recall_item_dict in agg_recall_item_dict.items():
            for i, score in recall_item_dict.items():
                recall_u_i_score_pair_list.append((u, i, score))
        recall_df = pd.DataFrame.from_records(recall_u_i_score_pair_list, columns=['user_id', 'item_id', 'sim'])
        return recall_df
    
    return agg_recall_item_dict


def get_multi_source_sim_dict_results(history_df, recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}):
    recall_sim_pair_dict = {}
    if 'item-cf' in recall_methods: 
        print('item-cf item-sim begin')
        item_sim_dict, _ = get_time_dir_aware_sim_item(history_df)
        recall_sim_pair_dict['item-cf'] = item_sim_dict
        print('item-cf item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))
        
    if  'bi-graph' in recall_methods:
        print('bi-graph item-sim begin')
        item_sim_dict, _ =  get_bi_sim_item(history_df)
        recall_sim_pair_dict['bi-graph'] =item_sim_dict
        print('bi-graph item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))
        
    if 'swing' in recall_methods:
        print('swing item-sim begin')
        item_sim_dict, _ =  swing(history_df)
        recall_sim_pair_dict['swing'] = item_sim_dict
        print('swing item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))
        
    if 'user-cf' in recall_methods:
        print('user-cf user-sim begin')
        user_sim_dict, _ =  get_sim_user(history_df)
        recall_sim_pair_dict['user-cf'] = user_sim_dict
        print('user-cf user-sim-pair done, pair_num={}'.format(len(user_sim_dict)))
        
    return recall_sim_pair_dict
        
        

def do_multi_recall_results(recall_sim_pair_dict, user_item_time_dict, target_user_ids=None, ret_type='df'):
    if target_user_ids is None:
        target_user_ids = user_item_time_dict.keys()
    
    recall_item_dict_list = []
    for name, sim_dict in recall_sim_pair_dict.items():
        # item-based
        if name in {'item-cf', 'bi-graph', 'swing'}:
            recall_item_dict = get_recall_results(sim_dict, user_item_time_dict, target_user_ids, item_based=True)
        else:
            recall_item_dict = get_recall_results(sim_dict, user_item_time_dict, target_user_ids, item_based=False)
        
        print('{} recall done, recall_user_num={}.'.format(name, len(recall_item_dict)))
        recall_item_dict_list.append(recall_item_dict)
        
    return agg_recall_results(recall_item_dict_list, is_norm=True, ret_type=ret_type)



In [20]:
# item-cf
# bi-graph
# user-cf
# item-cf
def agg_recall_results(recall_item_dict_list, is_norm=True, ret_type='tuple'):
    print('aggregate recall results begin....')
    agg_recall_item_dict = {}
    for recall_item_dict in recall_item_dict_list:
        if is_norm:
            recall_item_dict = norm_user_recall_item_dict(recall_item_dict)
        for u, recall_items in recall_item_dict.items():
            agg_recall_item_dict.setdefault(u, {})
            for i, score in recall_items:
                agg_recall_item_dict[u].setdefault(i, 0.0)
                agg_recall_item_dict[u][i] += score # 累加
                
    if ret_type == 'tuple':
        agg_recall_item_tuple_dict = {}
        for u, recall_item_dict in agg_recall_item_dict.items():
            sorted_recall_item_tuples = sorted(recall_item_dict.items(), key=lambda x:x[1], reverse=True)
            agg_recall_item_tuple_dict[u] = sorted_recall_item_tuples
        return agg_recall_item_tuple_dict
    
    if ret_type == 'df':
        recall_u_i_score_pair_list = []
        for u, recall_item_dict in agg_recall_item_dict.items():
            for i, score in recall_item_dict.items():
                recall_u_i_score_pair_list.append((u, i, score))
        recall_df = pd.DataFrame.from_records(recall_u_i_score_pair_list, columns=['user_id', 'item_id', 'sim'])
        return recall_df
    
    return agg_recall_item_dict


def get_multi_source_sim_dict_results(history_df, recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}):
    recall_sim_pair_dict = {}
    if 'item-cf' in recall_methods: 
        print('item-cf item-sim begin')
        item_sim_dict, _ = get_time_dir_aware_sim_item(history_df)
        recall_sim_pair_dict['item-cf'] = item_sim_dict
        print('item-cf item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))
        
    if  'bi-graph' in recall_methods:
        print('bi-graph item-sim begin')
        item_sim_dict, _ =  get_bi_sim_item(history_df)
        recall_sim_pair_dict['bi-graph'] =item_sim_dict
        print('bi-graph item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))
        
    if 'swing' in recall_methods:
        print('swing item-sim begin')
        item_sim_dict, _ =  swing(history_df)
        recall_sim_pair_dict['swing'] = item_sim_dict
        print('swing item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))
        
    if 'user-cf' in recall_methods:
        print('user-cf user-sim begin')
        user_sim_dict, _ =  get_sim_user(history_df)
        recall_sim_pair_dict['user-cf'] = user_sim_dict
        print('user-cf user-sim-pair done, pair_num={}'.format(len(user_sim_dict)))
        
    return recall_sim_pair_dict
        
def do_multi_recall_results(recall_sim_pair_dict, user_item_time_dict, target_user_ids=None, ret_type='df'):
    if target_user_ids is None:
        target_user_ids = user_item_time_dict.keys()
    
    recall_item_dict_list = []
    for name, sim_dict in recall_sim_pair_dict.items():
        # item-based
        if name in {'item-cf', 'bi-graph', 'swing'}:
            recall_item_dict = get_recall_results(sim_dict, user_item_time_dict, target_user_ids, item_based=True)
        else:
            recall_item_dict = get_recall_results(sim_dict, user_item_time_dict, target_user_ids, item_based=False)
        
        print('{} recall done, recall_user_num={}.'.format(name, len(recall_item_dict)))
        recall_item_dict_list.append(recall_item_dict)
        
    return agg_recall_results(recall_item_dict_list, is_norm=True, ret_type=ret_type)

### multi-processing recall

In [21]:
def get_multi_source_sim_dict_results_multi_processing(history_df, recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}, thread_num=4):
    def convert(history_df, input_q, result_q):
        while True:
            name = input_q.get()
            if 'item-cf' == name: 
                print('item-cf item-sim begin')
                item_sim_dict, _ = get_time_dir_aware_sim_item(history_df)
                result_q.put((name, item_sim_dict))
                print('item-cf item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

            elif  'bi-graph' == name:
                print('bi-graph item-sim begin')
                item_sim_dict, _ =  get_bi_sim_item(history_df)
                result_q.put((name, item_sim_dict))
                print('bi-graph item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

            elif 'swing' == name:
                print('swing item-sim begin')
                item_sim_dict, _ =  swing(history_df)
                result_q.put((name, item_sim_dict))
                print('swing item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

            elif 'user-cf' == name:
                print('user-cf user-sim begin')
                user_sim_dict, _ =  get_sim_user(history_df)
                result_q.put((name, user_sim_dict))
                print('user-cf user-sim-pair done, pair_num={}'.format(len(user_sim_dict)))
            input_q.task_done()
        
    from multiprocessing import Process, JoinableQueue, Queue   
    input_q = JoinableQueue()
    result_q = Queue()
        
    processes = []
    for name in recall_methods:
        input_q.put(name)
        processes.append(Process(target=convert, args=(history_df, input_q, result_q)))
        processes[-1].daemon = True
        processes[-1].start()
        
    input_q.join()
  
    recall_sim_pair_dict = {}
    while len(recall_sim_pair_dict) != len(recall_methods):
        name, sim_pair_dict = result_q.get()
        recall_sim_pair_dict[name] = sim_pair_dict  
    for p in processes:
        p.terminate()
        p.join()
        
    assert len(recall_sim_pair_dict) == len(recall_methods)
    return recall_sim_pair_dict

def do_multi_recall_results_multi_processing(recall_sim_pair_dict, user_item_time_dict, target_user_ids=None, 
                                                                ret_type='df', thread_num=4):
    from multiprocessing import Process, JoinableQueue, Queue
    def convert(user_item_time_dict, target_user_ids, item_based, input_q, result_q):
        while True:
            name, sim_dict = input_q.get()
            print('do recall for {}'.format(name))
            recall_item_dict = get_recall_results(sim_dict, user_item_time_dict, target_user_ids, item_based=item_based)
            result_q.put(recall_item_dict)
            print('{} recall done, recall_user_num={}.'.format(name, len(recall_item_dict)))
            input_q.task_done()
            
    input_q = JoinableQueue()
    result_q = Queue()
    
    if target_user_ids is None:
        target_user_ids = user_item_time_dict.keys()
        
    processes = []
    for name, sim_dict in recall_sim_pair_dict.items():
        item_based = True if name in {'item-cf', 'bi-graph', 'swing'} else False
        input_q.put((name, sim_dict))
        processes.append(Process(target=convert, args=(user_item_time_dict, target_user_ids, item_based, input_q, result_q)))
        processes[-1].daemon = True
        processes[-1].start()
        
    input_q.join()
  
    recall_item_dict_list = []
    while  len(recall_item_dict_list) != len(recall_sim_pair_dict):
        recall_item_dict = result_q.get()
        recall_item_dict_list.append(recall_item_dict)
        
    for p in processes:
        p.terminate()
        p.join()
        
    print(len(recall_item_dict_list))
    
    assert len(recall_item_dict_list) == len(recall_sim_pair_dict)
    return agg_recall_results(recall_item_dict_list, is_norm=True, ret_type=ret_type)

In [22]:
def make_item_sim_tuple(group_df):
    group_df = group_df.sort_values(by=['sim'], ascending=False)
    item_score_tuples = list(zip(group_df['item_id'], group_df['sim']))
    return item_score_tuples

def save_recall_df_as_user_tuples_dict(total_recom_df):
    
    save_path = os.path.join(drive_path, 'recall', 'online')
    !mkdir -p {save_path}
    
    for phase in range(now_phase+1):
        phase_df = total_recom_df[total_recom_df['phase'] == phase]
        phase_df = phase_df.groupby('user_id').apply(make_item_sim_tuple).reset_index().rename(columns={0: 'item_score_list'})
        phase_user_item_score_dict = dict(zip(phase_df['user_id'], phase_df['item_score_list']))
        pickle.dump(phase_user_item_score_dict, open(os.path.join(save_path, 'phase_{}.pkl'.format(phase)), 'wb'))
    
    pickle.dump(total_recom_df, open(os.path.join(save_path, 'total_recall_df.pkl'), 'wb'))
    
def read_recall_df():
    save_path = os.path.join(drive_path, 'recall', 'online')
    
    all_phase_user_item_score_dict = {}
    
    for phase in range(now_phase+1):
        phase_save_path = os.path.join(save_path, 'phase_{}.pkl'.format(phase))
        phase_user_item_score_dict = pickle.load(open(phase_save_path, 'rb'))
        all_phase_user_item_score_dict[phase] = phase_user_item_score_dict

    return all_phase_user_item_score_dict

### construct recall-training data

In [23]:
def get_history_and_last_click_df(click_df):
    click_df = click_df.sort_values(by=['user_id', 'time'])
    click_last_df = click_df.groupby('user_id').tail(1)

    # 用户只有1个点击时，history为空了，导致训练的时候这个用户不可见, 此时默认一下该用户泄露
    def hist_func(user_df):
        num = len(user_df)
        if num == 1:
            return user_df
        else:
            return user_df[:-1]

    click_history_df = click_df.groupby('user_id').apply(hist_func).reset_index(drop=True)
  
    return click_history_df, click_last_df

def sliding_obtain_training_df(c, is_silding_compute_sim=False, recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}):
    print('train_path={}, test_path={}'.format(train_path, test_path))
    
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,
                          names=['user_id', 'item_id', 'time'])
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,
                          names=['user_id', 'item_id', 'time'])
    all_click = click_train.append(click_test)

    click_history_df = all_click  # init
    total_step = 10
    step = 0
    
    # for validation
    compute_mode = 'once' if not is_silding_compute_sim else 'multi'
    
    save_training_path = os.path.join('training', mode, compute_mode, str(c))
    !mkdir -p {save_training_path}
    
    full_sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(click_history_df, recall_methods=recall_methods) 
    pickle.dump(full_sim_pair_dict, open(os.path.join(save_training_path, 'full_sim_pair_dict.pkl'), 'wb'))
    
    step_user_recall_item_dict = {}
    step_strategy_sim_pair_dict = {}
#     step_user_hist_item_time_dict = {}
    
    while step < total_step:
        print('step={}'.format(step))
        click_history_df, click_last_df = get_history_and_last_click_df(click_history_df)  # override click_history_df
        user_item_time_dict = get_user_item_time_dict(click_history_df)
        
        if is_silding_compute_sim:
            sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(click_history_df, recall_methods=recall_methods) # re-compute
        else:
            sim_pair_dict = full_sim_pair_dict
            
        user_recall_item_dict = do_multi_recall_results_multi_processing(sim_pair_dict, user_item_time_dict, ret_type='tuple')
        
        step_user_recall_item_dict[step] =  user_recall_item_dict
        if  is_silding_compute_sim:
            step_strategy_sim_pair_dict[step] = sim_pair_dict
         # step_user_hist_item_time_dict[step] = user_item_time_dict
        step += 1
    
    pickle.dump(step_user_recall_item_dict, open(os.path.join(save_training_path, 'step_user_recall_item_dict.pkl'), 'wb'))
    
    if  is_silding_compute_sim:
        pickle.dump(step_strategy_sim_pair_dict, open(os.path.join(save_training_path, 'step_strategy_sim_pair_dict.pkl'), 'wb'))
    
    # validation/test recall results based on full_sim_pair_dict
    # user-cf depend on sim-user history, so use all-click; test user history will not occur in train, so it's ok
    print('obtain validate/test recall data')
    all_user_item_dict = get_user_item_time_dict(all_click) 
    val_user_recall_item_dict = do_multi_recall_results_multi_processing(full_sim_pair_dict, 
                                                                    all_user_item_dict, 
                                                                    target_user_ids=click_test['user_id'].unique(), ret_type='tuple')
    
    pickle.dump(val_user_recall_item_dict, open(os.path.join(save_training_path, 'val_user_recall_item_dict.pkl'), 'wb'))

## recall-submit running

In [24]:
item_content_sim_dict = get_content_sim_item(item_feat_df, topk=200)

In [114]:
# time-aware-direction item-cf
recom_item = []
print("train_path={}, test_path={}".format(train_path, test_path))
whole_click = pd.DataFrame()

total_recom_df = pd.DataFrame()
for c in range(now_phase + 1):
    print('phase:', c)
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,
                              names=['user_id', 'item_id', 'time'])
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,
                              names=['user_id', 'item_id', 'time'])
    click_q_time = pd.read_csv(test_path + '/underexpose_test_qtime-{}.csv'.format(c), header=None,
                              names=['user_id', 'time'])

    all_click = click_train.append(click_test)
    whole_click = whole_click.append(all_click)
    
    recall_sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(all_click, recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}) 
    user_item_time_dict = get_user_item_time_dict(all_click)
    
    recom_df = do_multi_recall_results_multi_processing(recall_sim_pair_dict, user_item_time_dict, target_user_ids=click_q_time['user_id'].unique(), ret_type='df')
    recom_df['phase'] = c
    total_recom_df = total_recom_df.append(recom_df)
            
save_recall_df_as_user_tuples_dict(total_recom_df)

# find most popular items  
top50_click = whole_click['item_id'].value_counts().index[:50].values
top50_click = ','.join([str(i) for i in top50_click])
result = get_predict(total_recom_df, 'sim', top50_click)
result.to_csv(output_path + '/baseline_cf_v4.csv', index=False, header=None)

train_path=underexpose_train, test_path=underexpose_test
phase: 0
bi-graph item-sim begin
user-cf user-sim begin
swing item-sim begin
item-cf item-sim begin


 12%|█▏        | 2233/18505 [00:05<00:40, 401.19it/s]

user-cf user-sim-pair done, pair_num=18505


 34%|███▍      | 13997/40776 [00:08<00:14, 1903.66it/s]

swing item-sim-pair done, pair_num=40770


...

bi-graph recall done, recall_user_num=1798.


 93%|█████████▎| 1679/1798 [01:29<00:15,  7.52it/s]

item-cf recall done, recall_user_num=1798.


100%|██████████| 1798/1798 [01:33<00:00, 19.15it/s]


user-cf recall done, recall_user_num=1798.
4
aggregate recall results begin....
513000


## sliding construct training data

In [ ]:
for i in range(4, now_phase+1):
    sliding_obtain_training_df(i, is_silding_compute_sim=True)

train_path=underexpose_train, test_path=underexpose_test
swing item-sim begin
bi-graph item-sim begin
user-cf user-sim begin
item-cf item-sim begin


 10%|▉         | 4142/42840 [00:08<00:34, 1109.02it/s]

user-cf user-sim-pair done, pair_num=18618


 64%|██████▍   | 27444/42840 [00:38<00:26, 586.62it/s] 

In [ ]:
total_click

In [ ]:
ls

In [ ]:
for phase in range(now_phase+1):
    sliding_obtain_training_df(phase, is_compute_sim_once=True)

In [124]:
!ls training/online/once/0/

full_sim_pair_dict.pkl	step_user_recall_item_dict.pkl


## Evaluation

In [64]:
# coding=utf-8
from __future__ import division
from __future__ import print_function

import datetime
import json
import sys
import time
from collections import defaultdict

import numpy as np

# the higher scores, the better performance
def evaluate_each_phase(predictions, answers):
    list_item_degress = []
    for user_id in answers:
        item_id, item_degree = answers[user_id]
        list_item_degress.append(item_degree)
    list_item_degress.sort()
    median_item_degree = list_item_degress[len(list_item_degress) // 2]

    num_cases_full = 0.0
    ndcg_50_full = 0.0
    ndcg_50_half = 0.0
    num_cases_half = 0.0
    hitrate_50_full = 0.0
    hitrate_50_half = 0.0
    for user_id in answers:
        item_id, item_degree = answers[user_id]
        rank = 0
        while rank < 50 and predictions[user_id][rank] != item_id:
            rank += 1
        num_cases_full += 1.0
        if rank < 50:
            ndcg_50_full += 1.0 / np.log2(rank + 2.0)
            hitrate_50_full += 1.0
        if item_degree <= median_item_degree:
            num_cases_half += 1.0
            if rank < 50:
                ndcg_50_half += 1.0 / np.log2(rank + 2.0)
                hitrate_50_half += 1.0
    ndcg_50_full /= num_cases_full
    hitrate_50_full /= num_cases_full
    ndcg_50_half /= num_cases_half
    hitrate_50_half /= num_cases_half
    return np.array([ndcg_50_full, ndcg_50_half,
                     hitrate_50_full, hitrate_50_half], dtype=np.float32)

# submit_fname is the path to the file submitted by the participants.
# debias_track_answer.csv is the standard answer, which is not released.
def evaluate(submit_fname,
             answer_fname='debias_track_answer.csv', current_time=None):
    schedule_in_unix_time = [
        0,  # ........ 1970-01-01 08:00:00 (T=0)
        1586534399,  # 2020-04-10 23:59:59 (T=1)
        1587139199,  # 2020-04-17 23:59:59 (T=2)
        1587743999,  # 2020-04-24 23:59:59 (T=3)
        1588348799,  # 2020-05-01 23:59:59 (T=4)
        1588953599,  # 2020-05-08 23:59:59 (T=5)
        1589558399,  # 2020-05-15 23:59:59 (T=6)
        1590163199,  # 2020-05-22 23:59:59 (T=7)
        1590767999,  # 2020-05-29 23:59:59 (T=8)
        1591372799  # .2020-06-05 23:59:59 (T=9)
    ]
    assert len(schedule_in_unix_time) == 10
    for i in range(1, len(schedule_in_unix_time) - 1):
        # 604800 == one week
        assert schedule_in_unix_time[i] + 604800 == schedule_in_unix_time[i + 1]

    if current_time is None:
        current_time = int(time.time())
    print('current_time:', current_time)
    print('date_time:', datetime.datetime.fromtimestamp(current_time))
    current_phase = 0
    while (current_phase < 9) and (
            current_time > schedule_in_unix_time[current_phase + 1]):
        current_phase += 1
    print('current_phase:', current_phase)
  
    try:
        answers = [{} for _ in range(10)]
        with open(answer_fname, 'r') as fin:
            for line in fin:
                line = [int(x) for x in line.split(',')]
                phase_id, user_id, item_id, item_degree = line
                if mode == 'online':
                  assert user_id % 11 == phase_id
                # exactly one test case for each user_id
                answers[phase_id][user_id] = (item_id, item_degree)
    except Exception as e:
        return print('server-side error: answer file incorrect', e)

    try:
        predictions = {}
        with open(submit_fname, 'r') as fin:
            for line in fin:
                line = line.strip()
                if line == '':
                    continue
                line = line.split(',')
                user_id = int(line[0])
                if user_id in predictions:
                    return print('submitted duplicate user_ids')
                item_ids = [int(i) for i in line[1:]]
                if len(item_ids) != 50:
                    return print('each row need have 50 items')
                if len(set(item_ids)) != 50:
                    return print('each row need have 50 DISTINCT items')
                predictions[user_id] = item_ids
    except Exception as e:
        return print('submission not in correct format,e={}'.format(e))

    scores = np.zeros(4, dtype=np.float32)

    # The final winning teams will be decided based on phase T=7,8,9 only.
    # We thus fix the scores to 1.0 for phase 0,1,2,...,6 at the final stage.
    if current_phase >= 7:  # if at the final stage, i.e., T=7,8,9
        scores += 7.0  # then fix the scores to 1.0 for phase 0,1,2,...,6
    phase_beg = (7 if (current_phase >= 7) else 0)
    phase_end = current_phase + 1
    for phase_id in range(phase_beg, phase_end):
        for user_id in answers[phase_id]:
            if user_id not in predictions:
                return print('user_id %d of phase %d not in submission' % (
                        user_id, phase_id))
        try:
            # We sum the scores from all the phases, instead of averaging them.
            phase_score = evaluate_each_phase(predictions, answers[phase_id])
            print('phase_id={}, score={}'.format(phase_id, phase_score))
            scores += phase_score
        except Exception as e:
            return print('error occurred during evaluation, e={}'.format(e))
    
    print("score={},\nhitrate_50_full={},\nndcg_50_full={},\nhitrate_50_half={}, \nndcg_50_half={}".format(
        float(scores[0]), float(scores[2]), float(scores[0]), float(scores[3]), float(scores[1])
    ))
    return scores[0]
    # return report_score(
    #     stdout, score=float(scores[0]),
    #     ndcg_50_full=float(scores[0]), ndcg_50_half=float(scores[1]),
    #     hitrate_50_full=float(scores[2]), hitrate_50_half=float(scores[3]))

# FYI. You can create a fake answer file for validation based on this. For example,
# you can mask the latest ONE click made by each user in underexpose_test_click-T.csv,
# and use those masked clicks to create your own validation set, i.e.,
# a fake underexpose_test_qtime_with_answer-T.csv for validation.
def _create_answer_file_for_evaluation(output_answer_fname='debias_track_answer.csv'):
    train = train_path + '/underexpose_train_click-%d.csv'
    test = test_path + '/underexpose_test_click-%d.csv'

    # underexpose_test_qtime-T.csv contains only <user_id, time>
    # underexpose_test_qtime_with_answer-T.csv contains <user_id, item_id, time>
    answer = offline_answer_path + '/underexpose_test_qtime_with_answer-%d.csv'  # not released

    item_deg = defaultdict(lambda: 0)
    with open(output_answer_fname, 'w') as fout:
        for phase_id in range(now_phase+1):
            with open(train % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    item_deg[item_id] += 1
            with open(test % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    item_deg[item_id] += 1
            with open(answer % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    if mode == 'online':
                       assert user_id % 11 == phase_id
                    print(phase_id, user_id, item_id, item_deg[item_id],
                          sep=',', file=fout)
                    

# submit_fname is the path to the file submitted by the participants.
# debias_track_answer.csv is the standard answer, which is not released.
def evaluate_target_phase(submit_fname, target_phase, 
             answer_fname='debias_track_answer.csv', current_time=None):
    schedule_in_unix_time = [
        0,  # ........ 1970-01-01 08:00:00 (T=0)
        1586534399,  # 2020-04-10 23:59:59 (T=1)
        1587139199,  # 2020-04-17 23:59:59 (T=2)
        1587743999,  # 2020-04-24 23:59:59 (T=3)
        1588348799,  # 2020-05-01 23:59:59 (T=4)
        1588953599,  # 2020-05-08 23:59:59 (T=5)
        1589558399,  # 2020-05-15 23:59:59 (T=6)
        1590163199,  # 2020-05-22 23:59:59 (T=7)
        1590767999,  # 2020-05-29 23:59:59 (T=8)
        1591372799  # .2020-06-05 23:59:59 (T=9)
    ]
    assert len(schedule_in_unix_time) == 10
    for i in range(1, len(schedule_in_unix_time) - 1):
        # 604800 == one week
        assert schedule_in_unix_time[i] + 604800 == schedule_in_unix_time[i + 1]

    if current_time is None:
        current_time = int(time.time())
    print('current_time:', current_time)
    print('date_time:', datetime.datetime.fromtimestamp(current_time))
    current_phase = 0
    while (current_phase < 9) and (
            current_time > schedule_in_unix_time[current_phase + 1]):
        current_phase += 1
    print('current_phase:', current_phase)
  
    try:
        answers = [{} for _ in range(10)]
        with open(answer_fname, 'r') as fin:
            for line in fin:
                line = [int(x) for x in line.split(',')]
                phase_id, user_id, item_id, item_degree = line
                # assert user_id % 11 == phase_id
                # exactly one test case for each user_id
                answers[phase_id][user_id] = (item_id, item_degree)
    except Exception as e:
        return print('server-side error: answer file incorrect', e)

    try:
        predictions = {}
        with open(submit_fname, 'r') as fin:
            for line in fin:
                line = line.strip()
                if line == '':
                    continue
                line = line.split(',')
                user_id = int(line[0])
                if user_id in predictions:
                    return print('submitted duplicate user_ids')
                item_ids = [int(i) for i in line[1:]]
                if len(item_ids) != 50:
                    return print('each row need have 50 items')
                if len(set(item_ids)) != 50:
                    return print('each row need have 50 DISTINCT items')
                predictions[user_id] = item_ids
    except Exception as e:
        return print('submission not in correct format,e={}'.format(e))

    scores = np.zeros(4, dtype=np.float32)

    # The final winning teams will be decided based on phase T=7,8,9 only.
    # We thus fix the scores to 1.0 for phase 0,1,2,...,6 at the final stage.
    if current_phase >= 7:  # if at the final stage, i.e., T=7,8,9
        scores += 7.0  # then fix the scores to 1.0 for phase 0,1,2,...,6
    phase_beg = (7 if (current_phase >= 7) else 0)
    phase_end = current_phase + 1
    for phase_id in [target_phase]:
        for user_id in answers[phase_id]:
            if user_id not in predictions:
                return print('user_id %d of phase %d not in submission' % (
                        user_id, phase_id))
        try:
            # We sum the scores from all the phases, instead of averaging them.
            phase_score = evaluate_each_phase(predictions, answers[phase_id])
            print('phase_id={}, score={}'.format(phase_id, phase_score))
            scores += phase_score
        except Exception as e:
            return print('error occurred during evaluation e={}'.format(e))
    
    print("score={},\nhitrate_50_full={},\nndcg_50_full={},\nhitrate_50_half={}, \nndcg_50_half={}".format(
        float(scores[0]), float(scores[2]), float(scores[0]), float(scores[3]), float(scores[1])
    ))
    return scores[0]
    # return report_score(
    #     stdout, score=float(scores[0]),
    #     ndcg_50_full=float(scores[0]), ndcg_50_half=float(scores[1]),
    #     hitrate_50_full=float(scores[2]), hitrate_50_half=float(scores[3]))



In [78]:
# read answers(val data) in offline_answer_path, and output to output_answer_fname
_create_answer_file_for_evaluation(output_answer_fname=output_path +'/debias_track_answer.csv')

In [79]:
evaluate(submit_fname=output_path + "/baseline_cf_v4.csv", 
         answer_fname=output_path +'/debias_track_answer.csv') # topk

current_time: 1589113093
date_time: 2020-05-10 12:18:13
current_phase: 5
phase_id=0, score=[0.05561217 0.03750644 0.124375   0.08455115]
phase_id=1, score=[0.06618372 0.03779866 0.141875   0.08430913]
phase_id=2, score=[0.0620118  0.04202654 0.155625   0.10571081]
phase_id=3, score=[0.05628263 0.03226046 0.14       0.09307876]
phase_id=4, score=[0.06485526 0.05119485 0.15       0.10974106]
phase_id=5, score=[0.06123232 0.03808351 0.14125    0.08991495]
score=0.36617788672447205,
hitrate_50_full=0.8531249761581421,
ndcg_50_full=0.36617788672447205,
hitrate_50_half=0.567305862903595, 
ndcg_50_half=0.23887042701244354


0.3661779

In [79]:
evaluate(submit_fname=output_path + "/baseline_cf_v4.csv", 
         answer_fname=output_path +'/debias_track_answer.csv') # topk

current_time: 1589169665
date_time: 2020-05-11 04:01:05
current_phase: 5
phase_id=0, score=[0.05550259 0.03771673 0.12375    0.08559499]
phase_id=1, score=[0.06563789 0.03697544 0.1425     0.08430913]
phase_id=2, score=[0.0617016  0.0421126  0.153125   0.10571081]
phase_id=3, score=[0.0563484  0.03240005 0.139375   0.09307876]
phase_id=4, score=[0.06493273 0.05105051 0.150625   0.10974106]
phase_id=5, score=[0.06160757 0.03814993 0.143125   0.09113001]
score=0.36573076248168945,
hitrate_50_full=0.8525000214576721,
ndcg_50_full=0.36573076248168945,
hitrate_50_half=0.5695647597312927, 
ndcg_50_half=0.2384052723646164


0.36573076

In [ ]:
ls

In [84]:
!aws s3 cp sub_online/baseline_cf_v4.csv s3://mx-machine-learning/xuetaofeng/kdd/phase/5/baseline_cf_v4.csv

upload: sub_online/baseline_cf_v4.csv to s3://mx-machine-learning/xuetaofeng/kdd/phase/5/baseline_cf_v4.csv
upload: sub_online/baseline_cf_v4.csv to s3://mx-machine-learning/xuetaofeng/kdd/phase/5/baseline_cf_v4.csv
upload: sub_online/baseline_cf_v4.csv to s3://mx-machine-learning/xuetaofeng/kdd/phase/5/baseline_cf_v4.csv
upload: sub_online/baseline_cf_v4.csv to s3://mx-machine-learning/xuetaofeng/kdd/phase/5/baseline_cf_v4.csv


### phase

In [234]:
# read answers(val data) in offline_answer_path, and output to output_answer_fname
_create_answer_file_for_evaluation(output_answer_fname=output_path +'/debias_track_answer.csv')

In [65]:
evaluate_target_phase(output_path + "/baseline_recall_v1_phase_5.csv", 5, 
         answer_fname=output_path +'/debias_track_answer.csv')

current_time: 1589362639
date_time: 2020-05-13 09:37:19
current_phase: 5
phase_id=5, score=[0.06493513 0.03897732 0.14625    0.08991495]
score=0.0649351254105568,
hitrate_50_full=0.14624999463558197,
ndcg_50_full=0.0649351254105568,
hitrate_50_half=0.0899149477481842, 
ndcg_50_half=0.03897732496261597


0.064935125

In [113]:
evaluate_target_phase(output_path + "/baseline_recall_v1_phase_0.csv", 0, 
         answer_fname=output_path +'/debias_track_answer.csv')

current_time: 1589367947
date_time: 2020-05-13 11:05:47
current_phase: 5
phase_id=0, score=[0.05982477 0.04083483 0.133125   0.09290188]
score=0.05982477217912674,
hitrate_50_full=0.13312500715255737,
ndcg_50_full=0.05982477217912674,
hitrate_50_half=0.09290187805891037, 
ndcg_50_half=0.040834832936525345


0.059824772

In [114]:
evaluate_target_phase(output_path + "/baseline_ranking_v1_phase_0_offline.csv", 0, 
         answer_fname=output_path +'/debias_track_answer.csv') # online_total_click gbdt

current_time: 1589367953
date_time: 2020-05-13 11:05:53
current_phase: 5
phase_id=0, score=[0.06224673 0.04148717 0.14625    0.09812108]
score=0.06224673241376877,
hitrate_50_full=0.14624999463558197,
ndcg_50_full=0.06224673241376877,
hitrate_50_half=0.09812108427286148, 
ndcg_50_half=0.04148716852068901


0.062246732

In [91]:
evaluate_target_phase(output_path + "/baseline_ranking_v1_phase_0_offline.csv", 0, 
         answer_fname=output_path +'/debias_track_answer.csv') # gbdt+sliding, offline_total_click

current_time: 1589211630
date_time: 2020-05-11 15:40:30
current_phase: 5
phase_id=0, score=[0.06415642 0.03102796 0.14375    0.07724426]
score=0.0641564205288887,
hitrate_50_full=0.14374999701976776,
ndcg_50_full=0.0641564205288887,
hitrate_50_half=0.07724425941705704, 
ndcg_50_half=0.031027959659695625


0.06415642

In [217]:
evaluate_target_phase(output_path + "/baseline_ranking_v1_phase_5_offline.csv", 5, 
         answer_fname=output_path +'/debias_track_answer.csv') # gbdt + sliding + （no item statistic feat）

current_time: 1589348256
date_time: 2020-05-13 05:37:36
current_phase: 5
phase_id=5, score=[0.06714715 0.04146599 0.151875   0.09356014]
score=0.0671471506357193,
hitrate_50_full=0.15187500417232513,
ndcg_50_full=0.0671471506357193,
hitrate_50_half=0.09356014430522919, 
ndcg_50_half=0.041465990245342255


0.06714715

In [327]:
evaluate_target_phase(output_path + "/baseline_ranking_v1_phase_5_offline.csv", 5, 
         answer_fname=output_path +'/debias_track_answer.csv') # gbdt + sliding + fix interest degree bug

current_time: 1589352800
date_time: 2020-05-13 06:53:20
current_phase: 5
phase_id=5, score=[0.06716888 0.04090553 0.151875   0.09234508]
score=0.06716888397932053,
hitrate_50_full=0.15187500417232513,
ndcg_50_full=0.06716888397932053,
hitrate_50_half=0.09234508126974106, 
ndcg_50_half=0.040905531495809555


0.067168884

In [92]:
evaluate_target_phase(output_path + "/baseline_ranking_v1_phase_5_offline.csv", 5, 
         answer_fname=output_path +'/debias_track_answer.csv') # din + sliding + hist-feat

current_time: 1589365750
date_time: 2020-05-13 10:29:10
current_phase: 5
phase_id=5, score=[0.07039623 0.04510698 0.161875   0.10085054]
score=0.07039622962474823,
hitrate_50_full=0.16187499463558197,
ndcg_50_full=0.07039622962474823,
hitrate_50_half=0.10085054486989975, 
ndcg_50_half=0.04510698467493057


0.07039623

In [100]:
evaluate_target_phase(output_path + "/baseline_ranking_v1_phase_5_offline.csv", 5, 
         answer_fname=output_path +'/debias_track_answer.csv') # din + sliding + offline info

current_time: 1589366433
date_time: 2020-05-13 10:40:33
current_phase: 5
phase_id=5, score=[0.07039623 0.04510698 0.161875   0.10085054]
score=0.07039622962474823,
hitrate_50_full=0.16187499463558197,
ndcg_50_full=0.07039622962474823,
hitrate_50_half=0.10085054486989975, 
ndcg_50_half=0.04510698467493057


0.07039623

## ranking data

### organize recall feat

In [128]:
import time
t = (2020, 4, 10, 0, 0, 0, 0, 0, 0)
time_end = time.mktime(t)
max_day, max_hour, max_miniute = 7, 24, 60

def time_info(time_delta):
    timestamp = time_end * time_delta
    struct_time = time.gmtime(timestamp)
    day, hour, mini = struct_time.tm_wday+1, struct_time.tm_hour+1, struct_time.tm_min+1
    return (day, hour, mini)

def obtain_user_hist_feat(u, user_item_dict):
    user_hist_seq = [i for i, t in user_item_dict[u]]
    user_hist_day_seq, user_hist_hour_seq, user_hist_min_seq = zip(*[time_info(t) for i, t in user_item_dict[u]])
    return [user_hist_seq, list(user_hist_day_seq), list(user_hist_hour_seq), list(user_hist_min_seq)]
  
def organize_recall_feat_each_user(u, recall_items, user_item_dict, strategy_item_sim_dict, phase):
    user_hist_info = obtain_user_hist_feat(u, user_item_dict)
    
    # hist-item similarity with recall items
    hist_num = 3
    recall_items_sum_cf_sim2_hist = []
    recall_items_max_cf_sim2_hist = []
    recall_items_cnt_sim2_hist = []
    user_hist_items = user_item_dict[u][::-1][-hist_num:]

    for recall_i, rating in recall_items:
        if rating > 0:
            max_cf_sim2_hist = []
            sum_cf_sim2_hist = []
            cnt_sim2_hist = []
            for hist_i, t in user_hist_items:
                sum_sim_value = 0.0
                max_sim_value = 0.0
               
                for strategy, item_sim_dict in strategy_item_sim_dict.items():
                    strategy_sim_value = item_sim_dict.get(hist_i, {}).get(recall_i, 0.0) + item_sim_dict.get(recall_i, {}).get(hist_i, 0.0)
                    sum_sim_value += strategy_sim_value
                    max_sim_value = max(max_sim_value, strategy_sim_value)
                    
                cnt_sim_value = item_content_sim_dict.get(hist_i, {}).get(recall_i, 0.0) + item_content_sim_dict.get(recall_i, {}).get(hist_i, 0.0)
      
                sum_cf_sim2_hist.append(sum_sim_value)
                max_cf_sim2_hist.append(max_sim_value)
                cnt_sim2_hist.append(cnt_sim_value)

            while len(sum_cf_sim2_hist) < hist_num:
                sum_cf_sim2_hist.append(0.0)
                max_cf_sim2_hist.append(0.0)
                cnt_sim2_hist.append(0.0)
                
        else:
            sum_cf_sim2_hist = [0.0] * hist_num
            max_cf_sim2_hist = [0.0] * hist_num
            cnt_sim2_hist = [0.0] * hist_num

        recall_items_sum_cf_sim2_hist.append(sum_cf_sim2_hist)
        recall_items_max_cf_sim2_hist.append(max_cf_sim2_hist)
        recall_items_cnt_sim2_hist.append(cnt_sim2_hist)
    
    recom_item = []
    for item_rating, sum_cf_sim2_hist, max_cf_sim2_hist, cnt_sim2_hist in zip(recall_items, recall_items_sum_cf_sim2_hist, recall_items_max_cf_sim2_hist, recall_items_cnt_sim2_hist):
        recom_item.append([u, item_rating[0], item_rating[1], phase] + sum_cf_sim2_hist + max_cf_sim2_hist + \
                          cnt_sim2_hist + user_hist_info)
        
    return recom_item

def organize_recall_feat(recall_item_dict, user_item_hist_dict, item_sim_dict, phase):
    recom_columns = ['user_id', 'item_id', 'sim', 'phase'] + \
                      ['sum_sim2int_1', 'sum_sim2int_2', 'sum_sim2int_3'] + \
                             ['max_sim2int_1', 'max_sim2int_2', 'max_sim2int_3']  + \
                        ['cnt_sim2int_1', 'cnt_sim2int_2', 'cnt_sim2int_3'] + \
                          ['hist_item_id', 'hist_day_id', 'hist_hour_id', 'hist_minute_id']
    recom_item = []
    for u, recall_items in recall_item_dict.items():
        recom_item.extend(organize_recall_feat_each_user(u, recall_items, user_item_hist_dict, item_sim_dict, phase))

    recall_recom_df = pd.DataFrame(recom_item, columns=recom_columns)
    recall_recom_df['sim_rank_score'] = recall_recom_df.groupby('user_id')['sim'].rank(method='first', ascending=True) / topk_num
    
    return recall_recom_df

### organize label 

In [129]:
basic_columns = ['user_id','item_id', 'phase', 'label', ]
time_columns = ['time', 'day_id', 'hour_id', 'minute_id']
hist_columns = ['hist_item_id', 'hist_day_id', 'hist_hour_id', 'hist_minute_id',]
sim_columns = ['sim', 'sum_sim2int_1', 'sum_sim2int_2', 'sum_sim2int_3'] + \
                             ['max_sim2int_1', 'max_sim2int_2', 'max_sim2int_3', 'sim_rank_score']  + \
                              ['cnt_sim2int_1', 'cnt_sim2int_2', 'cnt_sim2int_3']

use_columns =  basic_columns + hist_columns + sim_columns + time_columns

def organize_label_interact_feat_df(click_last_df, click_last_recall_recom_df, phase, is_consider_cold_start=True):
    dfm_df = pd.merge(click_last_recall_recom_df, click_last_df[['user_id', 'item_id', 'time']], on=['user_id', 'item_id'], how='left') 
    dfm_df['label'] = dfm_df['time'].apply(lambda x: 0.0 if np.isnan(x) else 1.0) # time非空代表该click-item被召回了
    del dfm_df['time']

    # merge_time
    click_last_df['day_id'],  click_last_df['hour_id'], click_last_df['minute_id'] = zip(*click_last_df['time'].apply(time_info))
    dfm_df = pd.merge(dfm_df, click_last_df[['user_id', 'time', 'day_id', 'hour_id', 'minute_id']], on='user_id', how='left')


    # click_last_df里头有些用户的点击没有召回到，即：全部为负样本，导致下采样时，这些用户的负样本可能全被下采样掉了，导致这些用户id丢失；
    # item同理。用户真实点击的item可能没有召回到。
    dfm_df = downsample_by_user(dfm_df)
    dfm_df = dfm_df[use_columns]

    # cold_start_item直接泄露, 这些item可能在infer阶段被recall到，导致item_id缺失
    cold_start_items = set(click_last_df['item_id'].unique()) - set(dfm_df['item_id'].unique())
    if is_consider_cold_start and len(cold_start_items) > 0:
        click_last_cold_start_df = click_last_df[click_last_df['item_id'].isin(cold_start_items)]
        click_last_cold_start_df['label'] = 1.0
        click_last_cold_start_df['phase'] = phase
        for sim_col in sim_columns:
            mean_value = dfm_df[dfm_df['label'] == 1.0][sim_col].mean()
            print('sim_col={}, mean_value={}'.format(sim_col, mean_value))
            click_last_cold_start_df[sim_col] = mean_value
        click_last_cold_start_df = pd.merge(click_last_cold_start_df, dfm_df[['user_id',] + hist_columns], on='user_id', how='left')
    
        print('cold_start_item_num={}, hit_last_cold_start_df_num={}'.format(len(cold_start_items), len(click_last_cold_start_df)))
        dfm_df = dfm_df.append(click_last_cold_start_df[use_columns])

#     dfm_df = sim_process(dfm_df) # TODO, 移动到召回里头呢？
    return dfm_df

# def sim_process(dfm_df):
#     def norm_sim_by_user(sim_df):
#         min_sim = sim_df.min()
#         max_sim = sim_df.max()
#         if max_sim == min_sim:
#             sim_df = sim_df.apply(lambda sim: 1.0)
#         else:
#             sim_df = sim_df.apply(lambda sim: 1.0 * (sim - min_sim) / (max_sim - min_sim))
#         return sim_df
    
#     dfm_df['sim'] = dfm_df.groupby('user_id')['sim'].transform(norm_sim_by_user) 
#     dfm_df['exp_sim'] =  dfm_df['sim'].apply(lambda x: np.exp(x))
#     return dfm_df


def downsample_by_user(df, percent=10):
    '''
    percent:多数类别下采样的数量相对于少数类别样本数量的比例
    '''
    data_pos = df[df['label'] != 0]
    data_neg = df[df['label'] == 0]

    def group_neg_sample_func(group_df):
        total_neg_num = len(group_df)
        sample_num = max(int(total_neg_num * 0.002), 1) # 有些用户召回的数量不足, 取1个
        sample_num = min(sample_num, 5)
        return group_df.sample(n=sample_num, replace=True)

    data_u_neg = data_neg.groupby('user_id', group_keys=False).apply(group_neg_sample_func) # # 保证user全覆盖
    data_i_neg = data_neg.groupby('item_id', group_keys=False).apply(group_neg_sample_func) # 保证item全覆盖
    data_neg = data_u_neg.append(data_i_neg)
    data_neg = data_neg.sort_values(['user_id', 'sim']).drop_duplicates(['user_id', 'item_id'], keep='last')

    data = pd.concat([data_neg, data_pos], ignore_index=True)
    data = data.sample(frac=1.0)
    return data


### organize interact train/val data

1. 先获取steps的recall结果以及对应的 strategy_item_sim_dict

处理训练集：
2. 接着对每个step, 
      进行organize recall feat 
3. 对full_step_df进行organize label操作
 
处理验证集
1.  对验证集进行organize recall feat 
2. 对验证集也进行organize label操作

In [130]:
def organize_train_data(c, is_silding_compute_sim=False, online_is_eval=False):
    # 1. 获取recall的结果
    compute_mode = 'once' if not is_silding_compute_sim else 'multi'
    save_training_path = os.path.join('training', mode, compute_mode, str(c))
    
    print('train_path={}, test_path={}'.format(train_path, test_path))
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,
                          names=['user_id', 'item_id', 'time'])
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,
                          names=['user_id', 'item_id', 'time'])
    all_click = click_train.append(click_test)
    
    click_history_df = all_click  # init
    
    full_sim_pair_dict = pickle.load(open(os.path.join(save_training_path, 'full_sim_pair_dict.pkl'), 'rb'))
    step_user_recall_item_dict = pickle.load(open(os.path.join(save_training_path, 'step_user_recall_item_dict.pkl'), 'rb'))
    
    if  is_silding_compute_sim:
        step_strategy_sim_pair_dict = pickle.load(open(os.path.join(save_training_path, 'step_strategy_sim_pair_dict.pkl'), 'rb'))
    print('read recall data done...')
    
    step = 0
    total_step = 10
    train_full_df_list = []
    while step < total_step:
        print('step={} begin...'.format(step))
        click_history_df, click_last_df = get_history_and_last_click_df(click_history_df)  # override click_history_df
        user_item_time_dict = get_user_item_time_dict(click_history_df)
        
        user_recall_item_dict = step_user_recall_item_dict[step]
        strategy_sim_pair_dict =  step_strategy_sim_pair_dict[step] if is_silding_compute_sim else full_sim_pair_dict
        
         # organize recall interact feat
        click_last_recall_recom_df = organize_recall_feat(user_recall_item_dict, user_item_time_dict, strategy_sim_pair_dict, c)
        
        assert len(user_item_time_dict) == len(click_last_recall_recom_df['user_id'].unique()) == len(
            click_last_df['user_id'].unique())

        train_full_df = organize_label_interact_feat_df(click_last_df, click_last_recall_recom_df, c)
        print(train_full_df['label'].value_counts())
        train_full_df_list.append(train_full_df)
        step += 1
      
    if mode == 'offline':
        train_full_df = pd.concat(train_full_df_list, ignore_index=True)
    
        # valid data
        val_user_item_dict = get_user_item_time_dict(click_test) # click_test as history
        val_user_recall_item_dict = pickle.load(open(os.path.join(save_training_path, 'val_user_recall_item_dict.pkl'), 'rb'))

        phase_val_last_click_answer_df = pd.read_csv(offline_answer_path + '/underexpose_test_qtime_with_answer-{}.csv'.format(c), header=None, 
                                       names=['user_id', 'item_id', 'time']) 

        # organize recall interact feat
        phase_val_last_click_recall_recom_df = organize_recall_feat(val_user_recall_item_dict, val_user_item_dict, full_sim_pair_dict, c)
       
        val_full_df = organize_label_interact_feat_df(phase_val_last_click_answer_df, phase_val_last_click_recall_recom_df, phase, False)
        val_target_uids = phase_val_last_click_answer_df['user_id'].unique()
        
        save_train_val_path = os.path.join(save_training_path, 'train_val_label_target_id_data.pkl')
        pickle.dump([train_full_df, val_full_df, val_target_uids], open(save_train_val_path, 'wb'))
        
        return train_full_df, val_full_df, val_target_uids
    elif mode == 'online' and online_is_eval:
        print('online, use the last step as validation data')
        val_full_df = train_full_df_list[0]
        train_full_df = pd.concat(train_full_df_list[1:], ignore_index=True)
        val_target_uids = val_full_df.user_id.unique()
        
        save_train_val_path = os.path.join(save_training_path, 'train_val_label_target_id_data.pkl')
        pickle.dump([train_full_df, val_full_df, val_target_uids], open(save_train_val_path, 'wb'))
        
        return train_full_df, val_full_df, val_target_uids
    
    return train_full_df

In [131]:
def organize_train_data_multi_processing(c, is_silding_compute_sim=False, load_from_file=True):        
    # 1. 获取recall的结果
    compute_mode = 'once' if not is_silding_compute_sim else 'multi'
    save_training_path = os.path.join('training', mode, compute_mode, str(c))
    
    save_result_train_val_path = os.path.join(save_training_path, 'train_val_label_target_id_data.pkl')
    if load_from_file and os.path.exists(save_result_train_val_path):
        return pickle.load(open(save_result_train_val_path, 'rb'))
    
    print('train_path={}, test_path={}'.format(train_path, test_path))
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,
                          names=['user_id', 'item_id', 'time'])
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,
                          names=['user_id', 'item_id', 'time'])
    all_click = click_train.append(click_test)
    
    click_history_df = all_click  # init
    
    full_sim_pair_dict = pickle.load(open(os.path.join(save_training_path, 'full_sim_pair_dict.pkl'), 'rb'))
    step_user_recall_item_dict = pickle.load(open(os.path.join(save_training_path, 'step_user_recall_item_dict.pkl'), 'rb'))
    
    if  is_silding_compute_sim:
        step_strategy_sim_pair_dict = pickle.load(open(os.path.join(save_training_path, 'step_strategy_sim_pair_dict.pkl'), 'rb'))
    print('read recall data done...')
    
    step = 0
    total_step = 10
    
    from multiprocessing import Process, JoinableQueue, Queue   
        
    def convert(click_history_df, click_last_df, user_recall_item_dict, strategy_sim_pair_dict, input_q, result_q):
        step = input_q.get()
        print('step={} begin...'.format(step))
        user_item_time_dict = get_user_item_time_dict(click_history_df)
         # organize recall interact feat
        click_last_recall_recom_df = organize_recall_feat(user_recall_item_dict, user_item_time_dict, strategy_sim_pair_dict, c)
        
        assert len(user_item_time_dict) == len(click_last_recall_recom_df['user_id'].unique()) == len(
            click_last_df['user_id'].unique())

        train_full_df = organize_label_interact_feat_df(click_last_df, click_last_recall_recom_df, c)
        print(train_full_df['label'].value_counts())
        result_q.put(train_full_df)
        input_q.task_done()
        assert 'sim' in train_full_df.columns
        
    input_q = JoinableQueue()
    result_q = Queue()
        
    processes = []
    for step in range(total_step):
        input_q.put(step)
        click_history_df, click_last_df = get_history_and_last_click_df(click_history_df)  # override click_history_df
        user_recall_item_dict = step_user_recall_item_dict[step]
        strategy_sim_pair_dict =  step_strategy_sim_pair_dict[step] if is_silding_compute_sim else full_sim_pair_dict
        
        processes.append(Process(target=convert, args=(click_history_df, click_last_df, 
                                                                             user_recall_item_dict, strategy_sim_pair_dict,
                                                                             input_q, result_q)))
        processes[-1].daemon = True
        processes[-1].start()
        
    input_q.join()
    
    train_full_df_list = []
    while len(train_full_df_list) != total_step:
        train_full_df = result_q.get()
        train_full_df_list.append(train_full_df)
    
    for p in processes:
        p.terminate()
        p.join()
    
    print('obtain train data done....')
    
    assert len(train_full_df_list) == total_step
    
    if mode == 'offline':
        train_full_df = pd.concat(train_full_df_list, ignore_index=True)
        # valid data
        print('begin obtain validate data...')
        val_user_item_dict = get_user_item_time_dict(click_test) # click_test as history
        val_user_recall_item_dict = pickle.load(open(os.path.join(save_training_path, 'val_user_recall_item_dict.pkl'), 'rb'))

        phase_val_last_click_answer_df = pd.read_csv(offline_answer_path + '/underexpose_test_qtime_with_answer-{}.csv'.format(c), header=None, 
                                       names=['user_id', 'item_id', 'time']) 

        # organize recall interact feat
        phase_val_last_click_recall_recom_df = organize_recall_feat(val_user_recall_item_dict, val_user_item_dict, full_sim_pair_dict, c)
       
        val_full_df = organize_label_interact_feat_df(phase_val_last_click_answer_df, phase_val_last_click_recall_recom_df, phase, False)
        val_target_uids = phase_val_last_click_answer_df['user_id'].unique()
        
        save_train_val_path = os.path.join(save_training_path, 'train_val_label_target_id_data.pkl')
        pickle.dump([train_full_df, val_full_df, val_target_uids], open(save_train_val_path, 'wb'))
        
#         train_full_df = train_full_df.drop_duplicates(subset=['user_id', 'item_id'])
#         val_full_df = val_full_df.drop_duplicates(subset=['user_id', 'item_id'])
        
        return train_full_df, val_full_df, val_target_uids
    
#     else:
#         print('online, use the last step as validation data')
#         val_full_df = train_full_df_list[0]
#         train_full_df = pd.concat(train_full_df_list[1:], ignore_index=True)
#         val_target_uids = val_full_df.user_id.unique()
        
#         save_train_val_path = os.path.join(save_training_path, 'train_val_label_target_id_data.pkl')
#         pickle.dump([train_full_df, val_full_df, val_target_uids], open(save_train_val_path, 'wb'))
        
#         return train_full_df, val_full_df, val_target_uids

    return train_full_df

In [102]:
train_full_df_dict = {}
val_full_df_dict = {}
for i in [0]:
    train_full_df, val_full_df, val_target_uids = organize_train_data_multi_processing(i, is_silding_compute_sim=True)
    train_full_df_dict[i] = train_full_df
    val_full_df_dict[i] = val_full_df

In [132]:
online_train_full_df_dict = {}
for i in [0]:
    online_train_full_df = organize_train_data_multi_processing(i, is_silding_compute_sim=True, load_from_file=False)
    online_train_full_df_dict[i] = online_train_full_df

train_path=underexpose_train, test_path=underexpose_test
read recall data done...
step=0 begin...
step=1 begin...
step=2 begin...
step=3 begin...
step=4 begin...
step=5 begin...
step=6 begin...
step=7 begin...
step=8 begin...
step=9 begin...


/home/researcher/anaconda3/envs/deepctr/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/researcher/anaconda3/envs/deepctr/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


sim_col=sim, mean_value=0.9842633971087507


/home/researcher/anaconda3/envs/deepctr/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


sim_col=sum_sim2int_1, mean_value=0.11756902021679881
sim_col=sum_sim2int_2, mean_value=0.07886291107863397
sim_col=sum_sim2int_3, mean_value=0.061854033959642125
sim_col=max_sim2int_1, mean_value=0.08510828356831658
sim_col=max_sim2int_2, mean_value=0.057917302171004055
sim_col=max_sim2int_3, mean_value=0.04603264528715214
sim_col=sim_rank_score, mean_value=1.6084663250366011
sim_col=cnt_sim2int_1, mean_value=0.19339526005056557
sim_col=cnt_sim2int_2, mean_value=0.15198994807175395
sim_col=cnt_sim2int_3, mean_value=0.10730928265507364
cold_start_item_num=11, hit_last_cold_start_df_num=100
0.0    59857
1.0     4198
Name: label, dtype: int64


...

sim_col=sum_sim2int_1, mean_value=0.23809394290370756
sim_col=sum_sim2int_2, mean_value=0.15247126925640622
sim_col=sum_sim2int_3, mean_value=0.08082415407101143
sim_col=max_sim2int_1, mean_value=0.1588758521950985
sim_col=max_sim2int_2, mean_value=0.1019726782251352
sim_col=max_sim2int_3, mean_value=0.05716874652961455
sim_col=sim_rank_score, mean_value=1.476242371403657
sim_col=cnt_sim2int_1, mean_value=0.22943297151960076
sim_col=cnt_sim2int_2, mean_value=0.12745334934232125
sim_col=cnt_sim2int_3, mean_value=0.09113696285092321
cold_start_item_num=975, hit_last_cold_start_df_num=2901
0.0    57534
1.0     5195
Name: label, dtype: int64


/home/researcher/anaconda3/envs/deepctr/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/researcher/anaconda3/envs/deepctr/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


sim_col=sim, mean_value=0.9590249266619287


/home/researcher/anaconda3/envs/deepctr/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


sim_col=sum_sim2int_1, mean_value=0.12828821134235804
sim_col=sum_sim2int_2, mean_value=0.083967362779633
sim_col=sum_sim2int_3, mean_value=0.05030712181187723
sim_col=max_sim2int_1, mean_value=0.09054924261411398
sim_col=max_sim2int_2, mean_value=0.06117563943284279
sim_col=max_sim2int_3, mean_value=0.03790784997617621
sim_col=sim_rank_score, mean_value=1.5597291454730413
sim_col=cnt_sim2int_1, mean_value=0.20580672097533548
sim_col=cnt_sim2int_2, mean_value=0.1523468000236387
sim_col=cnt_sim2int_3, mean_value=0.10454719952503759
cold_start_item_num=47, hit_last_cold_start_df_num=276
0.0    59693
1.0     4208
Name: label, dtype: int64
obtain train data done....


In [133]:
online_train_full_df

,user_id,item_id,phase,label,hist_item_id,hist_day_id,hist_hour_id,hist_minute_id,sim,sum_sim2int_1,...,max_sim2int_2,max_sim2int_3,sim_rank_score,cnt_sim2int_1,cnt_sim2int_2,cnt_sim2int_3,time,day_id,hour_id,minute_id
46452,26482,108191,0,0.0,"[52995, 30085, 98315, 88947, 86572, 79446, 112...","[1, 1, 1, 1, 1, 3, 3, 3]","[24, 24, 24, 24, 24, 4, 4, 4]","[32, 33, 33, 34, 34, 9, 15, 16]",0.038626,0.000000,...,0.000000,0.000000,0.390000,0.000000,0.000000,0.000000,0.983887,3,4,19
45033,25576,52961,0,0.0,"[66644, 66479, 51239, 40013, 75798, 70567, 999...","[2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[22, 22, 22, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[31, 32, 37, 46, 47, 47, 48, 51, 52, 53, 55, 5...",0.035789,0.000000,...,0.000000,0.000000,1.020000,0.000000,0.000000,0.000000,0.983952,4,8,58
18801,10226,12877,0,0.0,[8191],[3],[23],[40],0.953142,0.236387,...,0.000000,0.000000,0.315000,0.000000,0.000000,0.000000,0.983931,3,23,40
58788,34742,67077,0,0.0,"[22855, 54429, 63843, 105832, 38863, 66881, 55...","[1, 1, 1, 1, 2, 2, 2, 2, 2, 3]","[9, 21, 21, 21, 2, 2, 3, 3, 3, 20]","[9, 5, 6, 7, 57, 60, 5, 8, 13, 13]",0.081007,0.000000,...,0.000000,0.163952,0.870000,0.000000,0.000000,0.000000,0.983924,3,20,30
50639,29136,50861,0,0.0,[29445],[2],[10],[34],0.136786,0.242803,...,0.000000,0.000000,0.045000,0.000000,0.000000,0.000000,0.983847,2,10,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,34625,18370,0,1.0,"[59093, 35118]","[3, 3]","[10, 10]","[22, 23]",0.959025,0.128288,...,0.061176,0.037908,1.559729,0.205807,0.152347,0.104547,0.983901,3,10,28
272,34868,756,0,1.0,"[59260, 20549]","[1, 1]","[1, 1]","[5, 6]",0.959025,0.128288,...,0.061176,0.037908,1.559729,0.205807,0.152347,0.104547,0.983771,1,1,11
273,34868,756,0,1.0,"[59260, 20549]","[1, 1]","[1, 1]","[5, 6]",0.959025,0.128288,...,0.061176,0.037908,1.559729,0.205807,0.152347,0.104547,0.983771,1,1,11
274,34868,756,0,1.0,"[59260, 20549]","[1, 1]","[1, 1]","[5, 6]",0.959025,0.128288,...,0.061176,0.037908,1.559729,0.205807,0.152347,0.104547,0.983771,1,1,11


In [32]:
train_target_phase_full_df = train_full_df_dict[5]
val_target_phase_full_df = val_full_df_dict[5]

In [166]:
online_val_full_df_dict[0]['label'].value_counts()

0.0    55805
1.0     6657
Name: label, dtype: int64

In [34]:
val_full_df['label'].value_counts()

0.0    47014
1.0      389
Name: label, dtype: int64

0.0    47014
1.0      389
Name: label, dtype: int64

0.0    47014
1.0      389
Name: label, dtype: int64

### organize raw user-item feat 

In [36]:
def process_item_feat(item_feat_df):
    processed_item_feat_df = item_feat_df.copy()
    txt_dense_feat = ['txt_embed_'+str(i) for i in range(128)] 
    img_dense_feat = ['img_embed_'+str(i) for i in range(128)]
    dense_feat = txt_dense_feat + img_dense_feat
    # norm
    txt_item_feat_np = processed_item_feat_df[txt_dense_feat].values
    img_item_feat_np = processed_item_feat_df[img_dense_feat].values
    txt_item_feat_np = txt_item_feat_np / np.linalg.norm(txt_item_feat_np, axis=1, keepdims=True)
    img_item_feat_np = img_item_feat_np / np.linalg.norm(img_item_feat_np, axis=1, keepdims=True)
    processed_item_feat_df[txt_dense_feat] = pd.DataFrame(txt_item_feat_np, columns=txt_dense_feat)
    processed_item_feat_df[img_dense_feat] = pd.DataFrame(img_item_feat_np, columns=img_dense_feat)

    # item_feat_dict = dict(zip(processed_item_feat_df['item_id'], processed_item_feat_df[dense_feat].values))
    return processed_item_feat_df, dense_feat

def process_user_feat(user_feat_df):
    # sparse encoder
    user_sparse_feat = ['age_level','gender','city_level']
    return user_feat_df, user_sparse_feat

In [37]:
def sparse_feat_fit(online_total_click):
    global feat_lbe_dict, item_raw_id2_idx_dict
    
    from sklearn.preprocessing import LabelEncoder, MinMaxScaler
    # sparse features one-hot
    feat_lbe_dict = {}
    for feat in sparse_feat:
        if feat in time_feat: continue
        lbe = LabelEncoder()
        lbe.fit(online_total_click[feat].astype(str))
        feat_lbe_dict[feat] = lbe
    
    item_raw_id2_idx_dict = dict(zip(feat_lbe_dict['item_id'].classes_, 
                     feat_lbe_dict['item_id'].transform(feat_lbe_dict['item_id'].classes_)+1, )) # 得到字典
    

def sparse_feat_transform(df):
    df['hist_item_id'] = df['hist_item_id'].apply(lambda seq: [item_raw_id2_idx_dict[str(x)] for x in seq])
    
    for hist_id in var_len_feat: 
        df[hist_id] = tf.keras.preprocessing.sequence.pad_sequences(df[hist_id], 
                                                  value=0, maxlen=max_seq_len, truncating='pre', padding='pre').tolist()
    for feat in sparse_feat:
        print(feat)
        if feat in time_feat: continue
        df[feat] = feat_lbe_dict[feat].transform(df[feat].astype(str))+1
    return df


In [81]:
def fillna(df, sparse_feat, dense_feat):
  for sp in sparse_feat:
    df[sp].fillna('-1', inplace=True)
    
  for ds in dense_feat:
    df[ds].fillna(0.0, inplace=True) # all_click_user_item_df[ds].mean()
  return df
  
def organize_user_item_feat(df, user_feat_df, item_feat_df, sparse_feat, dense_feat):
    full_user_df = pd.merge(df, user_feat_df, how='left', on='user_id')
    full_user_item_df = pd.merge(full_user_df, item_feat_df, how='left', on='item_id')
    full_user_item_df = fillna(full_user_item_df, sparse_feat, dense_feat)
    print('origin data done')
  
    # history interest
    full_user_item_df = obtain_user_hist_interest_feat(full_user_item_df, item_content_vec_dict)

    print('interest done')
    
    for key, file_name in zip(['item_id', 'user_id'], ['item_statistic.pkl', 'user_statistic.pkl']):
        count_df, first_time_df, last_time_df, \
                day_count_df, hour_count_df = pickle.load(open(os.path.join(drive_path, file_name), 'rb'))
        full_user_item_df = pd.merge(full_user_item_df, count_df, on=key, how='left')
        full_user_item_df = pd.merge(full_user_item_df, first_time_df, on=key, how='left')
        full_user_item_df = pd.merge(full_user_item_df, last_time_df, on=key, how='left')
        full_user_item_df = pd.merge(full_user_item_df, day_count_df, on=[key, 'day_id'],  how='left')
        full_user_item_df = pd.merge(full_user_item_df, hour_count_df, on=[key, 'hour_id'],  how='left')
        full_user_item_df['{}_day_count'.format(key)].fillna(0.0, inplace=True)
        full_user_item_df['{}_hour_count'.format(key)].fillna(0.0, inplace=True)
        
        full_user_item_df['diff_time'] = full_user_item_df['time'] -  full_user_item_df['item_id_first_time']

    print('statistic done')
    
    full_user_item_df = sparse_feat_transform(full_user_item_df)
    
    return full_user_item_df


def obtain_item_global_feat(total_click):
    total_click_df = total_click.copy()

    item_count_df = total_click_df.groupby('item_id', group_keys=False)['user_id'].count().reset_index().rename(columns={'user_id': 'item_degree'})
    item_count_info_dict = item_count_df['item_degree'].describe().to_dict()
    item_count_df['item_vs_degree_mean'] = item_count_df['item_degree'].apply(lambda x: 1.0 if x >= item_count_info_dict['mean'] else 0.0)
    item_count_df['item_vs_25_degree']  = item_count_df['item_degree'].apply(lambda x: 1.0 if x >= item_count_info_dict['25%'] else 0.0)
    item_count_df['item_vs_50_degree']  = item_count_df['item_degree'].apply(lambda x: 1.0 if x >= item_count_info_dict['50%'] else 0.0)
    item_count_df['item_vs_75_degree']  = item_count_df['item_degree'].apply(lambda x: 1.0 if x >= item_count_info_dict['75%'] else 0.0)
    
    
    item_last_time_df = total_click_df.groupby('item_id')['time'].max().reset_index().rename(columns={'time': 'item_id_last_time'})
    item_first_time_df = total_click_df.groupby('item_id')['time'].min().reset_index().rename(columns={'time': 'item_id_first_time'})
    
    total_click_df['day_id'],  total_click_df['hour_id'], total_click_df['minute_id'] = zip(*total_click_df['time'].apply(time_info))
    item_day_count_df =  total_click_df.groupby(['item_id', 'day_id']).size().reset_index().rename(columns={0: 'item_id_day_count'})
    item_hour_count_df =  total_click_df.groupby(['item_id', 'hour_id']).size().reset_index().rename(columns={0: 'item_id_hour_count'})
    
    pickle.dump([item_count_df, item_first_time_df, item_last_time_df, item_day_count_df, item_hour_count_df], 
                    open(os.path.join(drive_path, 'item_statistic.pkl'), 'wb'))
    
def obtain_user_global_feat(total_click):
    total_click_df = total_click.copy()   
    user_count_df = total_click_df.groupby('user_id', group_keys=False)['item_id'].count().reset_index().rename(columns={'item_id': 'user_degree'})
    user_count_info_dict = user_count_df['user_degree'].describe().to_dict()
    user_count_df['user_vs_degree_mean'] = user_count_df['user_degree'].apply(lambda x: 1.0 if x >= user_count_info_dict['mean'] else 0.0)
    user_count_df['user_vs_25_degree']  = user_count_df['user_degree'].apply(lambda x: 1.0 if x >= user_count_info_dict['25%'] else 0.0)
    user_count_df['user_vs_50_degree']  = user_count_df['user_degree'].apply(lambda x: 1.0 if x >= user_count_info_dict['50%'] else 0.0)
    user_count_df['user_vs_75_degree']  = user_count_df['user_degree'].apply(lambda x: 1.0 if x >= user_count_info_dict['75%'] else 0.0)
    
    user_last_time_df = total_click_df.groupby('user_id')['time'].max().reset_index().rename(columns={'time': 'user_id_last_time'})
    user_first_time_df = total_click_df.groupby('user_id')['time'].min().reset_index().rename(columns={'time': 'user_id_first_time'})
    
    
    total_click_df['day_id'],  total_click_df['hour_id'], total_click_df['minute_id'] = zip(*total_click_df['time'].apply(time_info))

    user_day_count_df =  total_click_df.groupby(['user_id', 'day_id']).size().reset_index().rename(columns={0: 'user_id_day_count'})
    user_hour_count_df =  total_click_df.groupby(['user_id', 'hour_id']).size().reset_index().rename(columns={0: 'user_id_hour_count'})
    
    pickle.dump([user_count_df, user_first_time_df, user_last_time_df, user_day_count_df, user_hour_count_df], 
                    open(os.path.join(drive_path, 'user_statistic.pkl'), 'wb'))
    

def obtain_user_hist_interest_feat(full_user_item_df, item_vec_dict):
    def weighted_agg_content(hist_item_id_list):

        weighted_content = np.zeros(128*2)
        hist_num = len(hist_item_id_list)
        for loc, i in enumerate(hist_item_id_list):
            loc_weight = (0.9**(hist_num-loc)) 
            if i in item_vec_dict:
                weighted_content += loc_weight*item_vec_dict[i]
        return weighted_content

    user_interest_vec = full_user_item_df['hist_item_id'].apply(weighted_agg_content).tolist()
    user_interest_df = pd.DataFrame(user_interest_vec, columns=['interest_'+col for col in item_dense_feat])
    
    full_user_item_df[user_interest_df.columns] = user_interest_df

    # begin compute degree
    target_item_vec = full_user_item_df[item_dense_feat].values
    user_interest_vec = np.array(user_interest_vec) 
    
    txt_interest_degree_array = target_item_vec[:, 0:128] * user_interest_vec[:, 0:128]
    txt_interest_degree_list = np.sum(txt_interest_degree_array, axis=1)
    full_user_item_df['txt_interest_degree'] = txt_interest_degree_list.tolist()
    
    img_interest_degree_array = target_item_vec[:, 128:] * user_interest_vec[:, 128:]
    img_interest_degree_list = np.sum(img_interest_degree_array, axis=1)
    full_user_item_df['img_interest_degree'] = img_interest_degree_list.tolist()
    
    full_user_item_df['interest_degree'] =  full_user_item_df['img_interest_degree']  + full_user_item_df['img_interest_degree'] 
    
    for f in ['interest_'+col for col in item_dense_feat]+['img_interest_degree', 'img_interest_degree', 'interest_degree']:
        full_user_item_df[f].fillna(0.0, inplace=True)
    print('obtain user dynamic feat done')
    
    def hist_2_target_cnt(hist_target_item_list, hist_no):
        target_item = hist_target_item_list[-1]
        if target_item not in item_content_vec_dict:
            return [0.0, 0.0, 0.0]

        hist_target_item_list = hist_target_item_list[: -1]

        if len(hist_target_item_list) >= hist_no:
            hist_item = hist_target_item_list[-hist_no]
            if hist_item in item_content_vec_dict:
                txt_cnt_sim = np.dot(item_content_vec_dict[target_item][0:128], item_content_vec_dict[hist_item][0:128])
                img_cnt_sim = np.dot(item_content_vec_dict[target_item][128:], item_content_vec_dict[hist_item][128:])
                return txt_cnt_sim, img_cnt_sim, txt_cnt_sim + img_cnt_sim

        return [0.0, 0.0, 0.0]

    hist_target_items_series = full_user_item_df['hist_item_id'] + full_user_item_df['item_id'].apply(lambda x:[x])
    full_user_item_df['txt_cnt_sim_last_1'], full_user_item_df['img_cnt_sim_last_1'], full_user_item_df['cnt_sim_last_1']  = zip(*hist_target_items_series.apply(lambda x: hist_2_target_cnt(x, 1)))
    full_user_item_df['txt_cnt_sim_last_2'], full_user_item_df['img_cnt_sim_last_2'], full_user_item_df['cnt_sim_last_2']  = zip(*hist_target_items_series.apply(lambda x: hist_2_target_cnt(x, 2)))
    full_user_item_df['txt_cnt_sim_last_3'], full_user_item_df['img_cnt_sim_last_3'], full_user_item_df['cnt_sim_last_3'] = zip(*hist_target_items_series.apply(lambda x: hist_2_target_cnt(x, 3)))
    return full_user_item_df

### running

In [103]:
target_phase = 0

In [50]:
processed_item_feat_df, item_dense_feat = process_item_feat(item_feat_df)
processed_user_feat_df, user_sparse_feat = process_user_feat(user_feat_df)
item_content_vec_dict = dict(zip(processed_item_feat_df['item_id'], processed_item_feat_df[item_dense_feat].values))

In [84]:
max_seq_len = 10
time_feat = ['day_id', 'hour_id'] #, 'minute_id']  # no need to sparse encoder
time_vocab_map = {'day_id': max_day, 'minute_id': max_miniute, 'hour_id': max_hour}

sparse_feat = ['user_id', 'item_id',] + time_feat # + user_sparse_feat
user_interest_dense_feat = ['interest_'+col for col in item_dense_feat] + ['interest_degree', 'txt_interest_degree', 'img_interest_degree',]
# sim_dense_feat =  ['sim', 'exp_sim', 'sim2int_1', 'sim2int_2', 'sim2int_3'] + ['cnt_sim2int_1', 'cnt_sim2int_2', 'cnt_sim2int_3'] # , 'sim_rank_score']
sim_dense_feat = ['sim', 'sum_sim2int_1', 'sum_sim2int_2', 'sum_sim2int_3'] + \
                             ['max_sim2int_1', 'max_sim2int_2', 'max_sim2int_3', 'sim_rank_score']  + \
                              ['cnt_sim2int_1', 'cnt_sim2int_2', 'cnt_sim2int_3']

hist_cnt_sim_feat = ['txt_cnt_sim_last_1', 'img_cnt_sim_last_1', 'cnt_sim_last_1'] + \
                            ['txt_cnt_sim_last_2', 'img_cnt_sim_last_2', 'cnt_sim_last_2'] + \
                            ['txt_cnt_sim_last_3', 'img_cnt_sim_last_3', 'cnt_sim_last_3'] 


item_degree_feat = ['item_degree', 'item_vs_degree_mean', 'item_vs_25_degree', 'item_vs_50_degree', 'item_vs_75_degree'] 
item_time_feat =  ['item_id_first_time', 'item_id_last_time', 'item_id_day_count', 'item_id_hour_count']
item_statistic_feat = item_degree_feat + item_time_feat

user_degree_feat = ['user_degree', 'user_vs_degree_mean', 'user_vs_25_degree', 'user_vs_50_degree', 'user_vs_75_degree'] 
user_time_feat =  ['user_id_first_time', 'user_id_last_time', 'user_id_day_count', 'user_id_hour_count']
user_statistic_feat = user_degree_feat + user_time_feat
                               
dense_feat = item_dense_feat  +  sim_dense_feat # + item_statistic_feat
var_len_feat = ['hist_item_id'] +  ['hist_{}'.format(feat) for feat in time_feat]

In [52]:
sparse_feat_fit(online_total_click)

In [134]:
# item statistic, global is useful ?
obtain_item_global_feat(online_total_click[online_total_click['phase'] == target_phase])
obtain_user_global_feat(online_total_click[online_total_click['phase'] == target_phase])

In [135]:
train_final_df = organize_user_item_feat(online_train_full_df_dict[target_phase], processed_user_feat_df, processed_item_feat_df, sparse_feat, dense_feat)

origin data done
obtain user dynamic feat done
interest done
statistic done
user_id
item_id
day_id
hour_id


In [106]:
val_final_df = organize_user_item_feat(val_full_df_dict[target_phase], processed_user_feat_df, processed_item_feat_df, sparse_feat, dense_feat)

origin data done
obtain user dynamic feat done
interest done
statistic done
user_id
item_id
day_id
hour_id


## ranking model

### lightgbm

In [86]:
import lightgbm as lgb
lgb_cols = dense_feat  + user_interest_dense_feat + hist_cnt_sim_feat # ['user_degree'] # ['item_count',]  #, 'first_time', 'last_time'] # item_statistic_feat + time_feat 

In [136]:
# 0.868919 -> 0.854(all_item_statistic_feat) -> 0.877 (item_count,first_time, last_time) -> 0.879 (item_count) -> item_degree_info (0.8826)
clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
        max_depth=-1, n_estimators=300, objective='binary',
        subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
        learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs=-1) # 300epoch, best, 0.882898, dense_feat  + hist_cnt_sim_feat user_interest_dense_feat 

clf.fit(train_final_df[lgb_cols],  train_final_df['label'], )
       # eval_set=[(val_final_df[lgb_cols], val_final_df['label'])],
     #   eval_metric='auc',   early_stopping_rounds=50) 

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=20, min_child_weight=50, min_split_gain=0.0,
               n_estimators=300, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=2018, reg_alpha=0.0, reg_lambda=1, silent=True,
               subsample=0.7, subsample_for_bin=200000, subsample_freq=1)

In [93]:
feat_importance = pd.Series(clf.feature_importances_, index=lgb_cols).sort_values(ascending=False).reset_index().rename(columns={'index':'feat', 0:'importance'})
feat_importance

,feat,importance
0,sim,787
1,sim_rank_score,736
2,cnt_sim_last_1,710
3,txt_cnt_sim_last_1,569
4,txt_embed_8,250
...,...,...
530,interest_txt_embed_56,0
531,interest_txt_embed_57,0
532,txt_embed_116,0
533,interest_txt_embed_63,0


In [137]:
feat_importance = pd.Series(clf.feature_importances_, index=lgb_cols).sort_values(ascending=False).reset_index().rename(columns={'index':'feat', 0:'importance'})
feat_importance

,feat,importance
0,sim,555
1,sim_rank_score,436
2,cnt_sim_last_1,401
3,txt_cnt_sim_last_1,276
4,interest_degree,143
...,...,...
530,txt_embed_107,1
531,img_embed_124,1
532,interest_img_embed_21,1
533,txt_embed_35,0


In [413]:
lgb_cols = feat_importance[feat_importance['importance'] > 0]['feat'].values

### DIN

In [330]:
HIDDEN_SIZE = (128, 128)
BATCH_SIZE = 256
EPOCH = 6
EMBED_DIM = 16
def generate_din_feature_columns(data, sparse_features, dense_features,):
    sparse_feature_columns = [SparseFeat(feat, vocabulary_size=len(feat_lbe_dict[feat].classes_)+1, embedding_dim=EMBED_DIM)
                              for i, feat in enumerate(sparse_features) if feat not in time_feat]

    sparse_feature_columns += [SparseFeat(feat, vocabulary_size=time_vocab_map[feat]+1, embedding_dim=EMBED_DIM)
                              for i, feat in enumerate(time_feat)]
    

    dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_features]

    var_feature_columns = [VarLenSparseFeat(SparseFeat('hist_item_id', vocabulary_size=len(feat_lbe_dict['item_id'].classes_)+1,
                                                       embedding_dim=EMBED_DIM,embedding_name='item_id'), 
                                            maxlen=max_seq_len)]

    var_feature_columns += [VarLenSparseFeat(SparseFeat('hist_{}'.format(feat), vocabulary_size=time_vocab_map[feat]+1,
                                              embedding_dim=EMBED_DIM,embedding_name=feat), maxlen=max_seq_len) 
                                                    for i, feat in enumerate(time_feat)]
    # DNN side
    dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns
    # FM side
    linear_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns
    # all feature names
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns + var_feature_columns)

    return feature_names, linear_feature_columns, dnn_feature_columns

In [340]:
feature_names, linear_feature_columns, dnn_feature_columns = generate_din_feature_columns(train_final_df, ['user_id', 'item_id'], dense_feat+user_interest_dense_feat)

In [357]:
train_input = {name: np.array(train_final_df[name].values.tolist()) for name in feature_names}
train_label = train_final_df['label'].values

In [358]:
val_input = {name: np.array(val_final_df[name].values.tolist()) for name in feature_names}
val_label = val_final_df['label'].values

In [359]:
behavior_feature_list = ['item_id'] #, 'day_id', 'hour_id',] #  'minute_id']
model = DIN(dnn_feature_columns, behavior_feature_list, dnn_hidden_units=HIDDEN_SIZE,
                dnn_dropout=0.5)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss="binary_crossentropy",
                  metrics=['binary_crossentropy', tf.keras.metrics.AUC()], )

The following Variables were used a Lambda layer's call (lambda_2), but
are not present in its tracked objects:
  <tf.Variable 'attention_sequence_pooling_layer_2/local_activation_unit_2/kernel:0' shape=(40, 1) dtype=float32>
  <tf.Variable 'attention_sequence_pooling_layer_2/local_activation_unit_2/bias:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


In [360]:
EPOCH = 1
model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=EPOCH,
          verbose=1, validation_data=(val_input, val_label), ) # 1:20目前最优结果, epoch. 0.8728

Train on 675364 samples, validate on 47403 samples


/home/researcher/anaconda3/envs/deepctr/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/researcher/anaconda3/envs/deepctr/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


675364/675364 [==============================] - 77s 114us/sample - loss: 0.2209 - binary_crossentropy: 0.2182 - auc_2: 0.8385 - val_loss: 0.1334 - val_binary_crossentropy: 0.1286 - val_auc_2: 0.8656


## generate recommend result

In [59]:
def get_recall_predict(infer_recall_df, phase):
    topk_fill_items  = online_top50_click if mode == 'online' else offline_top50_click
    result = get_predict(infer_recall_df, 'sim', topk_fill_items)
    result.to_csv(output_path + '/baseline_recall_v1_phase_{}.csv'.format(phase), index=False, header=None)
    return result

def get_rank_predict(dfm_infer_call_df, phase, infer_target_uids=None, rating_col='prob'):
    dfm_infer_call_df = dfm_infer_call_df.copy()
    fake_item = dfm_infer_call_df['item_id'].iloc[0]
    infer_users = set(dfm_infer_call_df['user_id'].unique())
  
    if infer_target_uids is None:
        infer_target_uids = infer_users

    for uid in infer_target_uids:
        if uid not in infer_users:
            print('uid={} not in infer_users'.format(uid))
            dfm_infer_call_df = dfm_infer_call_df.append({'user_id': uid, 'item_id': fake_item, 'prob': -10000}, ignore_index=True)

    dfm_infer_call_df['user_id'] = dfm_infer_call_df['user_id'].astype(int)
    dfm_infer_call_df['item_id'] = dfm_infer_call_df['item_id'].astype(int)
    
    topk_fill_items  = online_top50_click if mode == 'online' else offline_top50_click
    
    result = get_predict(dfm_infer_call_df, rating_col, topk_fill_items)
    result.to_csv(output_path + '/baseline_ranking_v1_phase_{}_{}.csv'.format(phase, mode), index=False, header=None)
    return dfm_infer_call_df, result

In [96]:
def infer_process(phase, load_from_file=True, is_sliding_compute_sim=True):
    print('train_path={}, test_path={}'.format(train_path, test_path))
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(phase), header=None,
                          names=['user_id', 'item_id', 'time'])
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(phase), header=None,
                          names=['user_id', 'item_id', 'time'])
    all_click = click_train.append(click_test)
    
    target_infer_user_df = pd.read_csv(test_path + '/underexpose_test_qtime-{}.csv'.format(phase), header=None, 
                                   names=['user_id', 'time'])  
    infer_user_item_time_dict = get_user_item_time_dict(all_click)
    
    if load_from_file:
        compute_mode = 'multi' if is_sliding_compute_sim else 'once'
        save_training_path = os.path.join('training', mode, compute_mode, str(phase)) 
        full_sim_pair_dict = pickle.load(open(os.path.join(save_training_path, 'full_sim_pair_dict.pkl'), 'rb'))
        infer_user_recall_item_dict = pickle.load(open(os.path.join(save_training_path, 'val_user_recall_item_dict.pkl'), 'rb'))
        print('load recall info from file done')
    else:
        full_sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(all_click, recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}) 
        infer_user_recall_item_dict = do_multi_recall_results_multi_processing(full_sim_pair_dict, infer_user_item_time_dict, 
                                                                     target_user_ids=target_infer_user_df['user_id'].unique(), ret_type='tuple')
    
    infer_recall_recom_df = organize_recall_feat(infer_user_recall_item_dict, infer_user_item_time_dict, 
                                                                  full_sim_pair_dict, phase)
#     infer_recall_recom_df = sim_process(infer_recall_recom_df) # TODO

       
    target_infer_user_df['day_id'],  target_infer_user_df['hour_id'], target_infer_user_df['minute_id'] = zip(*target_infer_user_df['time'].apply(time_info))
    infer_recall_recom_df = pd.merge(infer_recall_recom_df, target_infer_user_df[['user_id', 'time', 'day_id', 'hour_id', 'minute_id']], on='user_id', how='left')


    infer_final_df = organize_user_item_feat(infer_recall_recom_df, processed_user_feat_df, processed_item_feat_df,
                                          sparse_feat, dense_feat)
  
    return infer_recall_recom_df, infer_final_df

In [138]:
infer_recall_recom_df, infer_df = infer_process(target_phase, load_from_file=True, is_sliding_compute_sim=True)

train_path=underexpose_train, test_path=underexpose_test
load recall info from file done
origin data done
obtain user dynamic feat done
interest done
statistic done
user_id
item_id
day_id
hour_id


In [139]:
# lgb
lgb_infer_ans = clf.predict_proba(infer_df[lgb_cols],  axis=1)[:,1]
infer_recall_recom_df['prob'] = lgb_infer_ans

In [362]:
infer_input = {name: np.array(infer_df[name].values.tolist()) for name in feature_names}
din_infer_ans = model.predict(infer_input, batch_size=BATCH_SIZE)
infer_recall_recom_df['prob'] = din_infer_ans

In [149]:
infer_recall_recom_df['prob_sim'] = infer_recall_recom_df['prob'] + infer_recall_recom_df['sim']

In [214]:
get_recall_predict(infer_recall_recom_df, target_phase)

80000


,user_id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,2,36058,91413,67567,14234,19318,14233,91002,68704,68439,...,87564,49718,48669,93716,28376,15557,48653,25402,90445,91458
1,8,18307,1906,3760,39469,15029,26786,40318,5507,13948,...,82473,101412,33881,82650,21333,7012,112044,25945,237,6834
2,9,24232,8552,109651,40854,57849,36566,88062,114330,2611,...,62081,7939,83671,79533,10656,104625,78724,72498,100308,107297
3,17,25684,79967,80606,13285,37775,67210,51966,29918,31085,...,59323,55449,18984,53044,76342,86154,46596,22041,25540,46785
4,42,17491,40670,87254,103775,59783,35441,102082,13306,44534,...,73131,12061,86936,25313,41755,32361,41603,10566,42948,34742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,35338,8207,80101,12530,76806,60349,85661,68871,63511,45278,...,25367,89959,50277,63510,112860,111077,72079,58165,53276,67647
1596,35360,28966,81227,60010,13438,88172,80437,87571,49402,65560,...,70914,9613,52261,86029,94370,83599,92318,38114,71295,77767
1597,35372,70643,76806,93437,86473,90006,94634,36071,50844,88701,...,51321,69398,50958,94804,23097,113623,48186,89864,112860,77723
1598,35374,27267,107900,90052,36259,42248,95843,84091,36631,76623,...,45390,56555,65099,42976,56101,65437,45281,21175,94898,87111


In [99]:
infer_recall_df, result = get_rank_predict(infer_recall_recom_df, target_phase, rating_col='prob')
result

80000


,user_id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,2,36058,67567,14234,91413,19318,14233,91002,112197,68439,...,90988,16722,50151,68493,15557,93845,15556,103448,16875,90445
1,8,18307,39469,26786,15029,13948,228,40318,39921,34006,...,4492,106175,22123,276,33689,640,23773,34258,35140,33881
2,9,24232,109651,8552,40854,88062,21504,11170,57849,4340,...,7156,17696,35264,7939,9604,71671,10656,37580,54429,52766
3,17,13285,51966,80606,20848,29918,37775,25684,115990,32683,...,85426,80324,5141,106783,55449,11101,95508,4340,27657,714
4,42,87254,40670,13302,17491,13306,59783,103775,102082,23825,...,26149,75244,24411,64622,110891,111225,21580,41603,34934,23924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,35338,80101,8207,12530,60349,76806,85661,68871,87016,45278,...,25367,85900,63510,72079,67647,112860,50277,70391,58165,45578
1596,35360,28966,78438,7820,74204,80437,81227,94674,21426,99626,...,100444,94055,84094,47027,52261,75950,73369,795,115686,1363
1597,35372,76806,70643,90006,93437,94634,36071,50844,46491,63321,...,110845,23097,40989,53845,89864,116102,96020,98643,108205,74774
1598,35374,27267,36259,107900,90052,9499,95843,68931,52182,42248,...,80434,65099,41858,10914,97254,68886,81042,85613,41219,53367


In [140]:
get_recall_predict(infer_recall_recom_df, target_phase) # phase 0

83150


,user_id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,11,20389,10528,84026,79868,66955,83429,59862,21941,26711,...,25539,59382,79345,18552,65676,76676,11896,2538,103775,6264
1,22,108852,100827,85728,101900,80765,109803,86011,43263,10513,...,86836,30941,42910,25433,66475,58916,79511,6702,45772,7807
2,44,25356,88371,77047,25454,25306,95536,25197,11328,78709,...,109170,27179,108857,74261,60544,89554,93424,80464,70618,71152
3,55,95867,87011,60374,69312,100510,100940,40981,37819,77831,...,3506,6669,58966,54564,53785,82552,9555,4154,53403,59795
4,66,76806,85661,36071,42985,51182,50958,80679,78868,53845,...,67890,65476,25339,89524,27944,69398,93539,38289,63321,41604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,35321,109556,32148,91689,11902,96077,5307,3147,38330,40764,...,3428,4089,70245,81076,24212,104455,18513,3247,7394,76469
1659,35343,95312,11645,28880,33200,47905,64147,46639,63958,87020,...,11031,39717,69435,14148,83784,58096,30766,56746,29889,86127
1660,35354,100517,29975,98928,13623,22785,46670,26758,53056,16296,...,51289,71771,33697,37015,89124,82194,8410,17561,20131,102886
1661,35365,27816,67570,3970,82080,91392,68947,77740,92536,23318,...,16850,49491,65836,7347,72266,48266,59012,6171,48963,60538


In [141]:
# phase 0
infer_recall_df, result = get_rank_predict(infer_recall_recom_df, target_phase, rating_col='prob')
result

83150


,user_id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,11,79868,59255,21517,26711,84026,20389,66955,10528,3654,...,4453,40484,4971,52751,11595,68105,15844,11896,34630,6264
1,22,100827,108852,85728,109803,101900,98489,113764,80765,100703,...,113527,103417,105407,79511,570,40648,57042,43245,42910,81106
2,44,77047,88371,25356,25306,25454,95536,25197,25339,25178,...,109170,76806,27179,50844,113733,80464,70618,62532,71152,81717
3,55,95867,100510,40981,87011,100940,14626,35884,16725,91879,...,56638,4154,93328,9555,80865,33366,35878,18472,86054,20437
4,66,42985,51182,53845,22404,37665,76806,85661,36071,20115,...,73287,22383,32157,25178,85158,103204,79060,65476,66752,70691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,35321,32148,109556,5307,96077,11902,25334,3147,91689,63131,...,71220,6326,110141,12749,28741,12554,106031,4980,70245,9881
1659,35343,11645,28880,95312,46639,33200,97400,75564,25613,47905,...,29889,85458,111225,77322,1236,52119,71994,53566,32102,44195
1660,35354,100517,29975,46670,13623,22785,53056,26758,16296,5895,...,24025,62489,4844,32544,103202,6216,75986,20235,93516,23265
1661,35365,27816,91392,92538,92536,67570,92659,995,3970,93845,...,59012,27227,60538,82080,24978,96185,54019,93856,48266,75830


### submit one-phase

In [142]:
total_recom_df = pickle.load(open('sub_0513_cf_df_online.pkl', 'rb'))

In [143]:
total_recom_lgb_df = total_recom_df.copy()

In [144]:
total_recom_lgb_df = total_recom_lgb_df[total_recom_lgb_df['phase'] != 0]

In [145]:
set(infer_recall_recom_df['user_id'].unique())-set(total_recom_df[total_recom_df['phase'] == 0].user_id.unique())

set()

In [146]:
online_infer_recall_recom_df = infer_recall_recom_df[['user_id', 'item_id', 'prob']].rename(columns={'prob': 'sim'})
online_infer_recall_recom_df['phase'] = 0

In [147]:
online_infer_recall_recom_df

,user_id,item_id,sim,phase
0,1133,87420,0.503302,0
1,1133,47839,0.417899,0
2,1133,108699,0.278788,0
3,1133,4844,0.248607,0
4,1133,53692,0.246954,0
...,...,...,...,...
626986,20922,17522,0.028076,0
626987,20922,77119,0.011286,0
626988,20922,93858,0.013175,0
626989,20922,48807,0.012677,0


In [148]:
total_recom_lgb_df = total_recom_lgb_df.append(online_infer_recall_recom_df)

In [149]:
total_recom_lgb_df

,user_id,item_id,sim,phase
0,1,32360,4.000000,1
1,1,100116,0.663404,1
2,1,46297,1.337734,1
3,1,35247,0.956139,1
4,1,92349,0.886396,1
...,...,...,...,...
626986,20922,17522,0.028076,0
626987,20922,77119,0.011286,0
626988,20922,93858,0.013175,0
626989,20922,48807,0.012677,0


In [151]:
# find most popular items  
result = get_predict(total_recom_lgb_df, 'sim', online_top50_click)
result.to_csv(output_path + '/baseline_cf_ranking_0_v4.csv', index=False, header=None)

513000


In [152]:
output_path + '/baseline_cf_ranking_0_v4.csv'

'sub_online/baseline_cf_ranking_0_v4.csv'

In [153]:
!aws s3 cp 'sub_online/baseline_cf_ranking_0_v4.csv' s3://mx-machine-learning/xuetaofeng/kdd/phase/5/baseline_cf_ranking_0.csv

upload: sub_online/baseline_cf_ranking_0_v4.csv to s3://mx-machine-learning/xuetaofeng/kdd/phase/5/baseline_cf_ranking_0.csv


In [154]:
total_recom_lgb_df

,user_id,item_id,sim,phase
0,1,32360,4.000000,1
1,1,100116,0.663404,1
2,1,46297,1.337734,1
3,1,35247,0.956139,1
4,1,92349,0.886396,1
...,...,...,...,...
626986,20922,17522,0.028076,0
626987,20922,77119,0.011286,0
626988,20922,93858,0.013175,0
626989,20922,48807,0.012677,0
